In [4]:

#!pip install -r requirements.txt
#!pip install --upgrade jupyter
#!pip install --upgrade ipywidgets

In [1]:
import matplotlib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
import json
import csv
import time
import re
#import requests
import warnings
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_percentage_error, mean_absolute_error, balanced_accuracy_score
from sklearn.preprocessing import StandardScaler, RobustScaler, PowerTransformer, Normalizer, MinMaxScaler, MaxAbsScaler, QuantileTransformer
from sklearn.preprocessing import KBinsDiscretizer, LabelEncoder
from sklearn.ensemble import GradientBoostingRegressor, HistGradientBoostingRegressor
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import LinearSVR, LinearSVC
from sklearn.impute import SimpleImputer
from lightgbm import LGBMClassifier, LGBMRegressor
from xgboost import XGBClassifier, XGBRegressor
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from scipy.stats import randint, uniform, loguniform, zscore
from sklearn.utils import resample
import optuna
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report, make_scorer
import joblib
import logging
#warnings.filterwarnings('ignore')

# Configuración de la ruta del archivo CSV
file_path = 'docs/600k_tracks_anonimous.csv' 
logs_folder = "logs"
if not os.path.exists(logs_folder):
    os.makedirs(logs_folder)
log_file_prefix = os.path.join(logs_folder, "hyperparameters_")
scaler_config_file = os.path.join(logs_folder, "scaler_config.csv")

# Configuración para visualizaciones en Jupyter Notebook
%matplotlib inline
sns.set(style="whitegrid")
pd.set_option('display.max_columns', None)


df = pd.read_csv(file_path)
df.head()

,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,6,126903,0,0.645,0.4450,0,-13.338,1,0.4510,0.674,0.7440,0.151,0.127,104.851,3
1,0,98200,0,0.695,0.2630,0,-22.136,1,0.9570,0.797,0.0000,0.148,0.655,102.009,1
2,0,181640,0,0.434,0.1770,1,-21.180,1,0.0512,0.994,0.0218,0.212,0.457,130.418,5
3,0,176907,0,0.321,0.0946,7,-27.961,1,0.0504,0.995,0.9180,0.104,0.397,169.980,3
4,0,163080,0,0.402,0.1580,3,-16.900,0,0.0390,0.989,0.1300,0.311,0.196,103.220,4


In [3]:
for i in range(0, 10):
    num_popular_items = len(df[df['popularity'] > i*10])
    print(f"Número de elementos con más de {i*10} en popularidad:", num_popular_items)

Número de elementos con más de 0 en popularidad: 541982
Número de elementos con más de 10 en popularidad: 457045
Número de elementos con más de 20 en popularidad: 366684
Número de elementos con más de 30 en popularidad: 256400
Número de elementos con más de 40 en popularidad: 147681
Número de elementos con más de 50 en popularidad: 69603
Número de elementos con más de 60 en popularidad: 24868
Número de elementos con más de 70 en popularidad: 6213
Número de elementos con más de 80 en popularidad: 736
Número de elementos con más de 90 en popularidad: 37


In [4]:
subset_size = 200000
df_subset = df.sample(n=subset_size, random_state=42)
df_subset.head()

,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
241517,21,105015,0,0.890,0.6680,9,-10.044,0,0.0533,0.570,0.000002,0.0591,0.942,137.934,4
444213,37,234800,0,0.663,0.5510,1,-6.445,1,0.0280,0.697,0.000000,0.1800,0.339,128.992,4
106480,10,147733,0,0.376,0.0979,2,-18.280,1,0.0350,0.485,0.000577,0.1150,0.236,90.290,3
141137,29,194373,0,0.565,0.3270,5,-13.843,1,0.0295,0.781,0.000011,0.1310,0.556,103.256,3
586550,55,172500,0,0.582,0.9790,11,-2.492,0,0.0714,0.125,0.001520,0.3390,0.196,128.010,4


In [5]:
df = df_subset

features = df.drop('popularity', axis=1)

target = df['popularity']

# X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

X_train = features
y_train = target

imputer = SimpleImputer(strategy='mean')
X_train = imputer.fit_transform(X_train)

'''
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
'''

def print_regression_metrics(y_test, predictions):
    mse = mean_squared_error(y_test, predictions)
    r2 = r2_score(y_test, predictions)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_test, predictions)
#   mape = mean_absolute_percentage_error(y_test, predictions)
    mse_var = mse / np.var(y_test)

    print(f'Mean Squared Error: {mse}')
    print(f'R2 Score: {r2}')
    print(f'Root Mean Squared Error (RMSE): {rmse}')
    print(f'Mean Absolute Error (MAE): {mae}')
#   print(f'Mean Absolute Percentage Error (MAPE): {mape}')
    print(f'MSE to Variance Ratio: {mse_var}')


NameError: name 'df_subset' is not defined

In [2]:
def load_scaler_config(scaler_config_file):
    scaler_config = {}
    if os.path.exists(scaler_config_file):
        with open(scaler_config_file, 'r') as csvfile:
            reader = csv.reader(csvfile)
            for row in reader:
                if len(row) >= 2:
                    scaler_config[row[0]] = row[1]
                else:
                    print("Error: Hay lineas en blanco:", row)
    return scaler_config

def save_scaler_config(scaler_config, scaler_config_file):
    with open(scaler_config_file, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        for key, value in scaler_config.items():
            writer.writerow([key, value])

def define_pipeline(scaler, model):
    return Pipeline([
        ('scaler', scaler),
        ('model', model),
    ])

def write_to_log(model_name, scaler, best_params, best_score, elapsed_time):
    
    log_file = f"{log_file_prefix}_{model_name}_log.csv"
    try:
        with open(log_file, 'r') as f:
            first_line = f.readline()
            if not first_line:
                with open(log_file, 'a') as f:
                    f.write("Model_Name,Scaler,Best_Params,Best_Score,Elapsed_Time-minutes\n")
    except FileNotFoundError:
        with open(log_file, 'a') as f:
            f.write("Model_Name,Scaler,Best_Params,Best_Score,Elapsed_Time-minutes\n")
    
    with open(log_file, 'a') as f:
        f.write(f"{model_name},{scaler},\"{best_params}\",{best_score},{elapsed_time}\n")


def save_results_to_csv(df_results, model_name, search_algorithm):
    if not search_algorithm == RandomizedSearchCV:
        csv_file = f"{log_file_prefix}_{model_name}_{search_algorithm.__name__}_df.csv"
    else:
        csv_file = f"{log_file_prefix}_{model_name}_df.csv"
    
    if not os.path.isfile(csv_file):
        df_results.to_csv(csv_file, index=False)
    else:
        df_results.to_csv(csv_file, mode='a', header=False, index=False)
        

def hyperparameter_search(pipeline, params, scoring, search_algorithm, n_iter, X_train, y_train):
    if search_algorithm == RandomizedSearchCV:
        search = search_algorithm(pipeline, params, cv=5, scoring=scoring, refit=scoring[0], n_jobs=20, n_iter=n_iter)
    else:
        search = search_algorithm(pipeline, params, cv=5, scoring=scoring, refit=scoring[0], n_jobs=20)
    
    search.fit(X_train, y_train)
    return search


#Optuna evaluate
        
def evaluate_model(model, X_train, y_train, scaler):

    scoring = make_scorer(mean_squared_error, greater_is_better=False)
    pipeline = define_pipeline(scaler, model)
    cv_scores = cross_val_score(pipeline, X_train, y_train, cv=5, scoring=scoring)

    return cv_scores.mean()

def save_optuna_study(estudio, nombre_variable, carpeta="jobs"):

    if not os.path.exists(carpeta):
        os.makedirs(carpeta)

    ruta_archivo = os.path.join(carpeta, f"{nombre_variable}.pkl")
    joblib.dump(estudio, ruta_archivo)

def cargar_estudio_optuna(nombre_variable, carpeta="jobs"):

    ruta_archivo = os.path.join(carpeta, f"{nombre_variable}.pkl")

    if os.path.exists(ruta_archivo):
        study = joblib.load(ruta_archivo)
    else:
        study = optuna.create_study(direction='maximize')

    return study

def train_and_select_best_model(models, scalers, X_train, y_train, scoring=['neg_mean_squared_error', 'r2', 'explained_variance', 'neg_mean_absolute_error'], search_algorithm=RandomizedSearchCV, n_iter=20):
    all_df_results = []
    best_overall_model = None
    best_overall_score = float('-inf')

    scaler_config = load_scaler_config(scaler_config_file)

    for model_name, config in models.items():
        if not search_algorithm == optuna:
            model_name = config['model'].__class__.__name__
        best_score_for_model = float('-inf') 
        best_model_for_scaler = None
        best_scaler_for_model = None
        print(f"Training model: {model_name}")
        if model_name in scaler_config:
            scaler_list = [scaler for scaler in scalers if scaler.__class__.__name__ == scaler_config[model_name]]
        else:
            scaler_list = scalers
        
        for scaler in scaler_list: 
            start_time = time.time()
            if search_algorithm == optuna:
                objective_function = config
                study = cargar_estudio_optuna(model_name)
                study.optimize(lambda trial: objective_function(trial, X_train, y_train, scaler), n_trials=n_iter, n_jobs = 20)
                save_optuna_study(study, model_name)
                best_score = study.best_value
                best_params = study.best_params
                results = study.trials
                df_results = study.trials_dataframe()

            else: 
                pipeline = define_pipeline(scaler, config['model'])
                search = hyperparameter_search(pipeline, config['params'], scoring, search_algorithm, n_iter, X_train, y_train)
                best_score = search.best_score_
                best_params = search.best_params_
                results = search.cv_results_
                df_results = pd.DataFrame(results)

            if best_score > best_score_for_model:
                best_score_for_model = best_score
                
                best_model_for_scaler = model_name
                best_scaler_for_model = scaler.__class__.__name__

            elapsed_time = (time.time() - start_time)    
            metric_name = scoring[0]
            
            print(f'Best score for model {model_name} with {scaler.__class__.__name__} using {metric_name}: {best_score_for_model}')
            hours, remainder = divmod(elapsed_time, 3600)
            minutes, seconds = divmod(remainder, 60)

            # Formatear la salida
            elapsed_time_formatted = "{:02}:{:02}:{:02}".format(int(hours), int(minutes), int(seconds))

            print("Tiempo transcurrido:", elapsed_time_formatted)

            write_to_log(model_name, scaler, best_params, best_score_for_model, elapsed_time/60)
            save_results_to_csv(df_results, model_name, search_algorithm)
            all_df_results.append(df_results)

        if best_score_for_model > best_overall_score:
            best_overall_score = best_score_for_model
            best_overall_model = best_model_for_scaler

        if model_name not in scaler_config:
            scaler_config[model_name] = best_scaler_for_model
            save_scaler_config(scaler_config, scaler_config_file)

    print(f"Best overall model: {best_overall_model}")
    print(f"Best overall score: {best_overall_score}")

    combined_df = pd.concat(all_df_results, ignore_index=True)

    return combined_df

scalers = [StandardScaler(), RobustScaler(), PowerTransformer(), Normalizer(), MinMaxScaler(), MaxAbsScaler(), QuantileTransformer()]


In [8]:
def filter_and_sort(df, sort_by_column= 'mean_test_neg_mean_squared_error', ascending=True):
    param_model_columns = df.filter(like='param_model').columns

    aggregate_metric_columns = df.filter(regex=r'^(?!.*split).*$').columns

    selected_columns = list(set(param_model_columns) | set(aggregate_metric_columns))

    df_selected = df[selected_columns]

    df_sorted = df_selected.sort_values(by=sort_by_column, ascending=ascending)

    return df_sorted

In [ ]:
#log_file = os.path.join(logs_folder, "hyperparameters_linear_regression_log.csv")
models = {
    'LinearRegression': {
        'model': LinearRegression(),
        'params': {
            'model__fit_intercept': [True, False],
            'model__positive': [True, False],
        }
    },
    'Lasso': {
        'model': Lasso(),
        'params': {
            'model__alpha': loguniform(a=0.01, b=10),
            'model__selection': ['cyclic', 'random'],
            'model__tol': loguniform(a=0.0001, b=1),
            'model__max_iter': randint(100, 100000),
        }
    },
    'Ridge': {
        'model': Ridge(),
        'params': {
            'model__alpha': loguniform(a=0.01, b=10),
            'model__solver': ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga'],
            'model__tol': loguniform(a=0.0001, b=1),
            'model__max_iter': randint(100, 100000),
        }
    },
    'ElasticNet': {
        'model': ElasticNet(),
        'params': {
            'model__alpha': loguniform(a=0.01, b=10),
            'model__l1_ratio': uniform(loc=0.01, scale=0.99),
            'model__selection': ['cyclic', 'random'],
            'model__tol': loguniform(a=0.0001, b=1),
            'model__max_iter': randint(100, 100000),
        }
    },
}

best_model = train_and_select_best_model(models, scalers, X_train, y_train, n_iter=500)

In [47]:
#filter_and_sort(best_model, ascending=False)

#best_model

Defaulting to user installation because normal site-packages is not writeable


In [7]:
log_file_prefix = os.path.join(logs_folder, "optimization_final_")
def objective_linear_regression(trial, X_train, y_train, scaler):
    model = LinearRegression(
        fit_intercept=trial.suggest_categorical('fit_intercept', [True, False]),
        positive=trial.suggest_categorical('positive', [True, False])
    )
    return evaluate_model(model, X_train, y_train, scaler)

def objective_lasso(trial, X_train, y_train, scaler):
    model = Lasso(
        alpha=trial.suggest_float('alpha', 0.01, 0.03, log=True),
        selection=trial.suggest_categorical('selection', ['cyclic', 'random']),
        max_iter=trial.suggest_int('max_iter', 100, 100000)
    )
    return evaluate_model(model, X_train, y_train, scaler)

def objective_ridge(trial, X_train, y_train, scaler):
    model = Ridge(
        alpha=trial.suggest_float('alpha', 0.01, 10, log=True),
        solver=trial.suggest_categorical('solver', ['auto', 'svd', 'cholesky']),
        max_iter=trial.suggest_int('max_iter', 100, 100000)
    )
    return evaluate_model(model, X_train, y_train, scaler)

def objective_elasticnet(trial, X_train, y_train, scaler):
    model = ElasticNet(
        alpha=trial.suggest_float('alpha', 0.01, 0.03, log=True),
        l1_ratio=trial.suggest_float('l1_ratio', 0.8, 0.99),
        selection=trial.suggest_categorical('selection', ['cyclic','random']),
        max_iter=trial.suggest_int('max_iter', 100, 100000)
    )
    return evaluate_model(model, X_train, y_train, scaler)

models = {
    'LinearRegression': objective_linear_regression,
    'Lasso': objective_lasso,
    'Ridge': objective_ridge,
    'ElasticNet': objective_elasticnet,
}

best_model = train_and_select_best_model(models, scalers, X_train, y_train, search_algorithm=optuna, n_iter = 100)

[I 2024-05-20 07:09:38,181] A new study created in memory with name: no-name-c2e96973-b85f-4dc2-b666-6c2696417c55


Training model: LinearRegression


[I 2024-05-20 07:10:00,958] Trial 10 finished with value: -255.23057301094977 and parameters: {'fit_intercept': True, 'positive': False}. Best is trial 10 with value: -255.23057301094977.
[I 2024-05-20 07:10:02,491] Trial 8 finished with value: -277.0178986940723 and parameters: {'fit_intercept': True, 'positive': True}. Best is trial 10 with value: -255.23057301094977.
[I 2024-05-20 07:10:03,439] Trial 9 finished with value: -277.0502171105074 and parameters: {'fit_intercept': True, 'positive': True}. Best is trial 10 with value: -255.23057301094977.
[I 2024-05-20 07:10:03,526] Trial 13 finished with value: -255.2486988607709 and parameters: {'fit_intercept': True, 'positive': False}. Best is trial 10 with value: -255.23057301094977.
[I 2024-05-20 07:10:03,598] Trial 0 finished with value: -273.170719053114 and parameters: {'fit_intercept': False, 'positive': False}. Best is trial 10 with value: -255.23057301094977.
[I 2024-05-20 07:10:03,797] Trial 11 finished with value: -277.055156

Best score for model LinearRegression with QuantileTransformer using neg_mean_squared_error: -255.164797051993
Tiempo transcurrido: 00:02:07
Best overall model: LinearRegression
Best overall score: -255.164797051993


In [9]:
models = {
    'RandomForestRegressor': {
        'model': RandomForestRegressor(),
        'params': {
            'model__n_estimators': randint(20, 200),
            'model__max_depth': randint(4, 100),
            'model__max_features': ['sqrt', 'log2', None, 1.0],
        }
    },
    'HistGradientBoostingRegressor': { 
        'model': HistGradientBoostingRegressor(),
        'params': {
            'model__max_iter': randint(10, 400), #More or less equivalent to n_estimators, this model have no attribute n_estimators
            'model__max_depth': randint(4, 400), 
            'model__learning_rate': uniform(loc=0.01, scale=0.4),
            'model__max_leaf_nodes': randint(20,500), #None = no limit, if benefit of high numbers will be set to none. 
        }
    },
    'XGBRegressor': {
        'model': XGBRegressor(), #En la exploración inicial tardaba mucho
        'params': {
            'model__n_estimators': randint(20, 200),
            'model__max_depth': randint(4, 50),
            'model__learning_rate': uniform(loc=0.01, scale=0.4)
        }
    },
    'GradientBoostingRegressor': {
        'model': GradientBoostingRegressor(), #Tener en cuenta que este tarda muchisimo
        'params': {
            'model__n_estimators': randint(20, 200),
            'model__max_depth': randint(4, 50),
            'model__learning_rate': uniform(loc=0.01, scale=0.4),
            'model__max_features': ['sqrt', 'log2', None, 1.0],
        }
    },
}

best_model = train_and_select_best_model(models, scalers, X_train, y_train, n_iter = 100)

Training model: RandomForestRegressor
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.959218 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2568
[LightGBM] [Info] Number of data points in the train set: 80000, number of used features: 14
[LightGBM] [Info] Start training from score 27.615925
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.728442 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2569
[LightGBM] [Info] Number of data points in the train set: 80000, number of used features: 14
[LightGBM] [Info] Start training from score 27.536012
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.724967 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_co

/home/jsolis/.local/lib/python3.11/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.338789 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2569
[LightGBM] [Info] Number of data points in the train set: 80000, number of used features: 14
[LightGBM] [Info] Start training from score 27.536012
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.691670 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2569
[LightGBM] [Info] Number of data points in the train set: 80000, number of used features: 14
[LightGBM] [Info] Start training from score 27.597237
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.544474 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not eno

In [12]:
models = {
    'LGBMRegressor': {
        'model': LGBMRegressor(),
        'params': {
            'model__n_estimators': randint(60, 120),
            'model__max_depth': randint(10, 40),
            'model__learning_rate': uniform(loc=0.15, scale=0.2),
            'model__num_leaves': randint(20, 60),
        }
    },
}

best_model = train_and_select_best_model(models, scalers, X_train, y_train, n_iter = 50)

Training model: LGBMRegressor
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.144539 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2568
[LightGBM] [Info] Number of data points in the train set: 80000, number of used features: 14
[LightGBM] [Info] Start training from score 27.615925
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.049460 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2568
[LightGBM] [Info] Number of data points in the train set: 80000, number of used features: 14
[LightGBM] [Info] Start training from score 27.592887
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.049563 seconds.
You can set `force_row_wise=true` to remove the over

In [ ]:
models = {
    'LGBMRegressor': {
        'model': LGBMRegressor(),
        'params': {
            'model__n_estimators': [20,40,60,80,100,120,140,160,180,200],
            'model__max_depth': [None],
            'model__learning_rate': [0.1],
            'model__num_leaves': [31],
        }
    },
}
#No tardan demasiado salvo XGBRegressor y GradientBoostingRegressor
best_model = train_and_select_best_model(models, scalers, X_train, y_train, search_algorithm=GridSearchCV)

Training model: LGBMRegressor


In [ ]:
models = {
    'LGBMRegressor': {
        'model': LGBMRegressor(),
        'params': {
            'model__n_estimators': [100],
            'model__max_depth': [None, 5, 10, 15, 20, 25, 30, 35, 40],
            'model__learning_rate': [0.1],
            'model__num_leaves': [31],
        }
    },
}
#No tardan demasiado salvo XGBRegressor y GradientBoostingRegressor
best_model = train_and_select_best_model(models, scalers, X_train, y_train, search_algorithm=GridSearchCV)

In [ ]:
models = {
    'LGBMRegressor': {
        'model': LGBMRegressor(),
        'params': {
            'model__n_estimators': [100],
            'model__max_depth': [None],
            'model__learning_rate': [0.01,0.02,0.05,0.08,0.1,0.15,0.2,0.3,0.4],
            'model__num_leaves': [31],
        }
    },
}
#No tardan demasiado salvo XGBRegressor y GradientBoostingRegressor
best_model = train_and_select_best_model(models, scalers, X_train, y_train, search_algorithm=GridSearchCV)

In [ ]:
models = {
    'LGBMRegressor': {
        'model': LGBMRegressor(),
        'params': {
            'model__n_estimators': [100],
            'model__max_depth': [None],
            'model__learning_rate': [0.1],
            'model__num_leaves': [15,20,25,30,35,40,45],
        }
    },
}
#No tardan demasiado salvo XGBRegressor y GradientBoostingRegressor
best_model = train_and_select_best_model(models, scalers, X_train, y_train, search_algorithm=GridSearchCV)

In [ ]:
log_file_prefix = os.path.join(logs_folder, "optimization_final_")

def objective_rf(trial, X_train, y_train, scaler):
    model = RandomForestRegressor(
        n_estimators=trial.suggest_int('n_estimators', 150, 200),
        max_depth=trial.suggest_int('max_depth', 20, 100),
        max_features=trial.suggest_categorical('max_features', ['sqrt', 'log2'])
    )
    return evaluate_model(model, X_train, y_train, scaler)

def objective_gb(trial, X_train, y_train, scaler):
    model = GradientBoostingRegressor(
        n_estimators=trial.suggest_int('n_estimators', 79, 150),
        max_depth=trial.suggest_int('max_depth', 4, 24),
        learning_rate=trial.suggest_float('learning_rate', 0.01, 0.25),
        max_features=trial.suggest_categorical('max_features', ['sqrt', 'log2'])
    )
    return evaluate_model(model, X_train, y_train, scaler)

def objective_hgb(trial, X_train, y_train, scaler):
    model = HistGradientBoostingRegressor(
        max_iter=trial.suggest_int('max_iter', 200, 400),
        max_depth=trial.suggest_int('max_depth', 4, 400),
        learning_rate=trial.suggest_float('learning_rate', 0.01, 0.06),
        max_leaf_nodes=trial.suggest_int('max_leaf_nodes', 200, 500),
    )
    return evaluate_model(model, X_train, y_train, scaler)

def objective_lgbm(trial, X_train, y_train, scaler):
    model = LGBMRegressor(
        n_estimators=trial.suggest_int('n_estimators', 60, 120),
        max_depth=trial.suggest_int('max_depth', 10, 40),
        learning_rate=trial.suggest_float('learning_rate', 0.15, 0.19),
        num_leaves=trial.suggest_int('num_leaves', 20, 60),
    )
    return evaluate_model(model, X_train, y_train, scaler)

def objective_xgb(trial, X_train, y_train, scaler):
    model = XGBRegressor(
        n_estimators=trial.suggest_int('n_estimators', 70, 200),
        max_depth=trial.suggest_int('max_depth', 5, 15),
        learning_rate=trial.suggest_float('learning_rate', 0.029, 0.2),
    )
    return evaluate_model(model, X_train, y_train, scaler)


models = {
    'RandomForestRegressor': objective_rf,
    'GradientBoostingRegressor': objective_gb,
    'LGBMRegressor': objective_lgbm,
    'XGBRegressor': objective_xgb,
    'HistGradientBoostingRegressor': objective_hgb,
}
#No tardan demasiado salvo XGBRegressor y GradientBoostingRegressor

for i in range (5):
    best_model = train_and_select_best_model(models, scalers, X_train, y_train, search_algorithm=optuna, n_iter=100)

Training model: RandomForestRegressor


[I 2024-05-22 08:22:59,866] Trial 420 finished with value: -212.54696772782853 and parameters: {'n_estimators': 197, 'max_depth': 27, 'max_features': 'sqrt'}. Best is trial 188 with value: -212.3436201568626.
[I 2024-05-22 08:23:29,177] Trial 431 finished with value: -212.44717990694716 and parameters: {'n_estimators': 197, 'max_depth': 30, 'max_features': 'sqrt'}. Best is trial 188 with value: -212.3436201568626.
[I 2024-05-22 08:23:45,607] Trial 426 finished with value: -212.4744517401703 and parameters: {'n_estimators': 197, 'max_depth': 35, 'max_features': 'sqrt'}. Best is trial 188 with value: -212.3436201568626.
[I 2024-05-22 08:23:47,816] Trial 427 finished with value: -212.57554070499577 and parameters: {'n_estimators': 197, 'max_depth': 35, 'max_features': 'sqrt'}. Best is trial 188 with value: -212.3436201568626.
[I 2024-05-22 08:24:12,757] Trial 433 finished with value: -212.52039362582087 and parameters: {'n_estimators': 197, 'max_depth': 34, 'max_features': 'sqrt'}. Best i

Best score for model RandomForestRegressor with RobustScaler using neg_mean_squared_error: -212.3094079394437
Tiempo transcurrido: 01:08:34
Best overall model: RandomForestRegressor
Best overall score: -212.3094079394437
Training model: GradientBoostingRegressor


[I 2024-05-22 09:29:19,286] Trial 436 finished with value: -213.17494287876374 and parameters: {'n_estimators': 150, 'max_depth': 15, 'learning_rate': 0.04214116747365079, 'max_features': 'log2'}. Best is trial 221 with value: -212.2309611893566.
[I 2024-05-22 09:29:35,926] Trial 430 finished with value: -213.0980722393725 and parameters: {'n_estimators': 150, 'max_depth': 15, 'learning_rate': 0.044565538086089165, 'max_features': 'log2'}. Best is trial 221 with value: -212.2309611893566.
[I 2024-05-22 09:29:38,058] Trial 428 finished with value: -212.41386963907252 and parameters: {'n_estimators': 150, 'max_depth': 15, 'learning_rate': 0.02952027608679814, 'max_features': 'log2'}. Best is trial 221 with value: -212.2309611893566.
[I 2024-05-22 09:30:43,903] Trial 438 finished with value: -213.11559575071828 and parameters: {'n_estimators': 150, 'max_depth': 15, 'learning_rate': 0.04309731133378818, 'max_features': 'log2'}. Best is trial 221 with value: -212.2309611893566.
[I 2024-05-2

Best score for model GradientBoostingRegressor with MaxAbsScaler using neg_mean_squared_error: -212.2309611893566
Tiempo transcurrido: 00:58:11
Best overall model: GradientBoostingRegressor
Best overall score: -212.2309611893566
Training model: LGBMRegressor
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.013023 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2568
[LightGBM] [Info] Number of data points in the train set: 160000, number of used features: 14
[LightGBM] [Info] Start training from score 27.608469
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.010562 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2568
[LightGBM] [Info] Number of data points in the train set: 160000, number of used features: 14
[LightGBM] [Info] Start training from score

...

[I 2024-05-22 12:03:24,425] Trial 804 finished with value: -214.11100061124202 and parameters: {'max_iter': 389, 'max_depth': 24, 'learning_rate': 0.022917418805179227, 'max_leaf_nodes': 343}. Best is trial 592 with value: -213.30612338032725.
[I 2024-05-22 12:11:08,866] Trial 811 finished with value: -213.78445949586035 and parameters: {'max_iter': 387, 'max_depth': 21, 'learning_rate': 0.02326254499374278, 'max_leaf_nodes': 477}. Best is trial 592 with value: -213.30612338032725.
[I 2024-05-22 12:18:13,429] Trial 807 finished with value: -213.66637330028988 and parameters: {'max_iter': 388, 'max_depth': 23, 'learning_rate': 0.023331885508409936, 'max_leaf_nodes': 477}. Best is trial 592 with value: -213.30612338032725.
[I 2024-05-22 12:18:26,366] Trial 813 finished with value: -213.77727427980872 and parameters: {'max_iter': 390, 'max_depth': 24, 'learning_rate': 0.02298146404370946, 'max_leaf_nodes': 478}. Best is trial 592 with value: -213.30612338032725.
[I 2024-05-22 12:18:30,320

Best score for model HistGradientBoostingRegressor with QuantileTransformer using neg_mean_squared_error: -213.30612338032725
Tiempo transcurrido: 06:25:40
Best overall model: HistGradientBoostingRegressor
Best overall score: -213.30612338032725
Training model: RandomForestRegressor


[I 2024-05-22 17:38:46,468] Trial 534 finished with value: -212.77821279385134 and parameters: {'n_estimators': 199, 'max_depth': 23, 'max_features': 'sqrt'}. Best is trial 486 with value: -212.3094079394437.
[I 2024-05-22 17:40:38,649] Trial 535 finished with value: -212.86258454226973 and parameters: {'n_estimators': 199, 'max_depth': 23, 'max_features': 'sqrt'}. Best is trial 486 with value: -212.3094079394437.
[I 2024-05-22 17:40:41,844] Trial 524 finished with value: -212.7660671572027 and parameters: {'n_estimators': 199, 'max_depth': 23, 'max_features': 'sqrt'}. Best is trial 486 with value: -212.3094079394437.
[I 2024-05-22 17:40:46,735] Trial 520 finished with value: -212.80091720378505 and parameters: {'n_estimators': 199, 'max_depth': 23, 'max_features': 'sqrt'}. Best is trial 486 with value: -212.3094079394437.
[I 2024-05-22 17:41:03,680] Trial 538 finished with value: -212.92115471191642 and parameters: {'n_estimators': 199, 'max_depth': 23, 'max_features': 'sqrt'}. Best i

Best score for model RandomForestRegressor with RobustScaler using neg_mean_squared_error: -212.3094079394437
Tiempo transcurrido: 01:19:04
Best overall model: RandomForestRegressor
Best overall score: -212.3094079394437
Training model: GradientBoostingRegressor


[I 2024-05-22 18:50:33,285] Trial 538 finished with value: -226.91593693641212 and parameters: {'n_estimators': 130, 'max_depth': 6, 'learning_rate': 0.028660240148136135, 'max_features': 'log2'}. Best is trial 221 with value: -212.2309611893566.
[I 2024-05-22 18:58:21,380] Trial 525 finished with value: -212.95162261740842 and parameters: {'n_estimators': 125, 'max_depth': 17, 'learning_rate': 0.02768730651299751, 'max_features': 'log2'}. Best is trial 221 with value: -212.2309611893566.
[I 2024-05-22 18:58:24,739] Trial 529 finished with value: -213.1269526483872 and parameters: {'n_estimators': 125, 'max_depth': 17, 'learning_rate': 0.028263391754259643, 'max_features': 'log2'}. Best is trial 221 with value: -212.2309611893566.
[I 2024-05-22 18:59:01,557] Trial 537 finished with value: -212.57343798443168 and parameters: {'n_estimators': 149, 'max_depth': 15, 'learning_rate': 0.02781790170164376, 'max_features': 'log2'}. Best is trial 221 with value: -212.2309611893566.
[I 2024-05-2

Para KNN

In [28]:
models = {
    'KNeighborsRegressor': {
        'model': KNeighborsRegressor(),
        'params': {
            'model__n_neighbors': randint(1, 10),
            'model__weights': ['uniform', 'distance'],
            'model__algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
            'model__p': [1,2,3], #p = 1, this is equivalent to using manhattan_distance (l1), and euclidean_distance (l2) for p = 2. For arbitrary p, minkowski_distance (l_p) is used.
            'model__leaf_size': randint(10, 50)
        }
    },
}

best_model = train_and_select_best_model(models, scalers, X_train, y_train, n_iter = 250)

Training model: KNeighborsRegressor
Best score for model KNeighborsRegressor with QuantileTransformer using neg_mean_squared_error: -243.13826899132295
Tiempo transcurrido: 04:47:41
Best overall model: KNeighborsRegressor
Best overall score: -243.13826899132295


In [ ]:
log_file_prefix = os.path.join(logs_folder, "optimization_final_")

def objective_kneighbors(trial, X_train, y_train, scaler):
    model = KNeighborsRegressor(
        n_neighbors=trial.suggest_int('n_neighbors', 65, 75),
        weights=trial.suggest_categorical('weights', ['distance']),
        algorithm=trial.suggest_categorical('algorithm', ['brute']),
        p=trial.suggest_int('p', 1, 1),
    )
    return evaluate_model(model, X_train, y_train, scaler)

models = {
    'KNeighborsRegressor': objective_kneighbors,
}

best_model = train_and_select_best_model(models, scalers, X_train, y_train, search_algorithm=optuna, n_iter=10)

[I 2024-05-20 09:40:59,432] A new study created in memory with name: no-name-17071a01-a73e-4a9f-bea7-99d90c284165


Training model: KNeighborsRegressor


[I 2024-05-20 09:42:04,117] Trial 0 finished with value: -223.6788915148532 and parameters: {'n_neighbors': 69, 'weights': 'distance', 'algorithm': 'brute', 'p': 1}. Best is trial 0 with value: -223.6788915148532.
[I 2024-05-20 09:43:09,073] Trial 1 finished with value: -223.74648239523896 and parameters: {'n_neighbors': 67, 'weights': 'distance', 'algorithm': 'brute', 'p': 1}. Best is trial 0 with value: -223.6788915148532.
[I 2024-05-20 09:44:13,389] Trial 2 finished with value: -223.6728649834685 and parameters: {'n_neighbors': 67, 'weights': 'distance', 'algorithm': 'brute', 'p': 1}. Best is trial 2 with value: -223.6728649834685.
[I 2024-05-20 09:45:17,152] Trial 3 finished with value: -223.73030714166893 and parameters: {'n_neighbors': 68, 'weights': 'distance', 'algorithm': 'brute', 'p': 1}. Best is trial 2 with value: -223.6728649834685.


In [7]:
log_file_prefix = os.path.join(logs_folder, "optimization_final_grid_")
models = {
    'KNeighborsRegressor': {
        'model': KNeighborsRegressor(),
        'params': {
            'model__n_neighbors': [9,10,11,12,13,14,15,20,25,30,35,40,45,50,55,60,65,70,75,80,85,90,95,100],
            'model__weights': ['distance'],
            'model__algorithm': ['brute'],
            'model__p': [1], #p = 1, this is equivalent to using manhattan_distance (l1), and euclidean_distance (l2) for p = 2. For arbitrary p, minkowski_distance (l_p) is used.
        }
    },
}
#No tarda mucho
best_model = train_and_select_best_model(models, scalers, X_train, y_train, n_iter = 100, search_algorithm=GridSearchCV)

Training model: KNeighborsRegressor


/home/jsolis/.local/lib/python3.11/site-packages/sklearn/model_selection/_search.py:318: UserWarning: The total space of parameters 29 is smaller than n_iter=100. Running 29 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best score for model KNeighborsRegressor with QuantileTransformer using neg_mean_squared_error: -223.6822729993964
Tiempo transcurrido: 00:15:55
Best overall model: KNeighborsRegressor
Best overall score: -223.6822729993964


In [ ]:
models = {
    'LinearSVR': {
        'model': LinearSVR(),
        'params': {
            'model__C': uniform(loc=0.1, scale=0.9),  # Regularization parameter. The strength of the regularization is inversely proportional to C. Must be strictly positive. 
            'model__tol': loguniform(a=0.0001, b=1),
            'model__max_iter': randint(500, 100000),
            'model__dual': ['auto'],
        }
    },
}

best_model = train_and_select_best_model(models, scalers, X_train, y_train, n_iter = 200)

In [6]:
log_file_prefix = os.path.join(logs_folder, "optimization_final_")
def objective_lsvr(trial, X_train, y_train, scaler):
    model = LinearSVR(
        dual=trial.suggest_categorical('dual', ['auto']),
        C=trial.suggest_float('C', 0.1, 0.7),
        max_iter=trial.suggest_int('max_iter', 10000, 100000),
    )
    return evaluate_model(model, X_train, y_train, scaler)

models = {
    'LinearSVR': objective_lsvr,
}

for i in range(5):
    best_model = train_and_select_best_model(models, scalers, X_train, y_train, search_algorithm=optuna, n_iter=100)

[I 2024-05-23 12:39:17,355] A new study created in memory with name: no-name-7a0ef6d1-0e3c-449a-9af7-9c63307fd8c2


Training model: LinearSVR


[I 2024-05-23 12:39:33,083] Trial 15 finished with value: -259.77355997425195 and parameters: {'dual': 'auto', 'C': 0.16259383939865057, 'max_iter': 72621}. Best is trial 15 with value: -259.77355997425195.
[I 2024-05-23 12:39:34,518] Trial 16 finished with value: -259.926307235438 and parameters: {'dual': 'auto', 'C': 0.5365464630890151, 'max_iter': 98793}. Best is trial 15 with value: -259.77355997425195.
[I 2024-05-23 12:39:36,471] Trial 11 finished with value: -259.8296937664488 and parameters: {'dual': 'auto', 'C': 0.3745800440993111, 'max_iter': 44199}. Best is trial 15 with value: -259.77355997425195.
[I 2024-05-23 12:39:36,719] Trial 13 finished with value: -259.89865849623703 and parameters: {'dual': 'auto', 'C': 0.2997982994114879, 'max_iter': 59449}. Best is trial 15 with value: -259.77355997425195.
[I 2024-05-23 12:39:37,715] Trial 0 finished with value: -260.0578076695186 and parameters: {'dual': 'auto', 'C': 0.4563782580600674, 'max_iter': 38538}. Best is trial 15 with va

Best score for model LinearSVR with QuantileTransformer using neg_mean_squared_error: -259.4944342007386
Tiempo transcurrido: 00:01:53
Best overall model: LinearSVR
Best overall score: -259.4944342007386
Training model: LinearSVR


[I 2024-05-23 12:41:30,593] Trial 117 finished with value: -259.9231303336077 and parameters: {'dual': 'auto', 'C': 0.19335219527512104, 'max_iter': 97965}. Best is trial 88 with value: -259.4944342007386.
[I 2024-05-23 12:41:31,437] Trial 105 finished with value: -259.7290070300907 and parameters: {'dual': 'auto', 'C': 0.16458789406269866, 'max_iter': 97821}. Best is trial 88 with value: -259.4944342007386.
[I 2024-05-23 12:41:32,436] Trial 116 finished with value: -259.86072664818204 and parameters: {'dual': 'auto', 'C': 0.1847736611527309, 'max_iter': 97725}. Best is trial 88 with value: -259.4944342007386.
[I 2024-05-23 12:41:32,679] Trial 104 finished with value: -259.84226677454546 and parameters: {'dual': 'auto', 'C': 0.18886709601823495, 'max_iter': 97306}. Best is trial 88 with value: -259.4944342007386.
[I 2024-05-23 12:41:32,769] Trial 109 finished with value: -259.785211257959 and parameters: {'dual': 'auto', 'C': 0.18898231246579486, 'max_iter': 98006}. Best is trial 88 wi

...

Best score for model LinearSVR with QuantileTransformer using neg_mean_squared_error: -259.4944342007386
Tiempo transcurrido: 00:01:58
Best overall model: LinearSVR
Best overall score: -259.4944342007386
Training model: LinearSVR


[I 2024-05-23 12:45:26,986] Trial 313 finished with value: -259.71501865646394 and parameters: {'dual': 'auto', 'C': 0.10242385869026614, 'max_iter': 62599}. Best is trial 88 with value: -259.4944342007386.
[I 2024-05-23 12:45:27,068] Trial 303 finished with value: -259.76238357015654 and parameters: {'dual': 'auto', 'C': 0.10107485642068473, 'max_iter': 96970}. Best is trial 88 with value: -259.4944342007386.
[I 2024-05-23 12:45:27,586] Trial 304 finished with value: -259.63993666931026 and parameters: {'dual': 'auto', 'C': 0.12218548846568886, 'max_iter': 57130}. Best is trial 88 with value: -259.4944342007386.
[I 2024-05-23 12:45:27,613] Trial 316 finished with value: -259.74625275993515 and parameters: {'dual': 'auto', 'C': 0.126063070882965, 'max_iter': 62940}. Best is trial 88 with value: -259.4944342007386.
[I 2024-05-23 12:45:28,161] Trial 308 finished with value: -259.83842641036244 and parameters: {'dual': 'auto', 'C': 0.12531195647013602, 'max_iter': 58619}. Best is trial 88

Best score for model LinearSVR with QuantileTransformer using neg_mean_squared_error: -259.4944342007386
Tiempo transcurrido: 00:01:58
Best overall model: LinearSVR
Best overall score: -259.4944342007386
Training model: LinearSVR


[I 2024-05-23 12:47:27,023] Trial 417 finished with value: -259.6178524016749 and parameters: {'dual': 'auto', 'C': 0.10137188764287366, 'max_iter': 13949}. Best is trial 88 with value: -259.4944342007386.
[I 2024-05-23 12:47:27,445] Trial 412 finished with value: -259.70113400161546 and parameters: {'dual': 'auto', 'C': 0.10215688256758015, 'max_iter': 48390}. Best is trial 88 with value: -259.4944342007386.
[I 2024-05-23 12:47:27,751] Trial 415 finished with value: -259.834935909545 and parameters: {'dual': 'auto', 'C': 0.10575961324225495, 'max_iter': 79136}. Best is trial 88 with value: -259.4944342007386.
[I 2024-05-23 12:47:27,794] Trial 400 finished with value: -259.68627336880155 and parameters: {'dual': 'auto', 'C': 0.10016922260069114, 'max_iter': 39144}. Best is trial 88 with value: -259.4944342007386.
[I 2024-05-23 12:47:27,812] Trial 414 finished with value: -259.87693653935264 and parameters: {'dual': 'auto', 'C': 0.10273327673479203, 'max_iter': 25734}. Best is trial 88 

Best score for model LinearSVR with QuantileTransformer using neg_mean_squared_error: -259.4944342007386
Tiempo transcurrido: 00:02:02
Best overall model: LinearSVR
Best overall score: -259.4944342007386


Remove outliers auxiliar function

In [ ]:
def remove_outliers(X, y, threshold=3):

    z_scores = np.abs(zscore(X))

    indices = (z_scores < threshold).all(axis=1)

    X_filtered = X[indices]
    y_filtered = y[indices]

    return X_filtered, y_filtered

**Clasificación**

In [13]:
logging.basicConfig(filename='logs/discretization_log.txt', level=logging.INFO)

# Una estratificación agresiva no es recomendable, 
# no obstante a objeto de la experimentación, 
# necesitamos una función para lidiar con ello.

# Optuna evaluate adapted to classification metrics
def evaluate_model(model, X_train, y_train, scaler):

    scoring = make_scorer(balanced_accuracy_score, greater_is_better=True)
    pipeline = define_pipeline(scaler, model)
    cv_scores = cross_val_score(pipeline, X_train, y_train, cv=5, scoring=scoring)

    return cv_scores.mean()

# Oversample minority classes to ensure at least two samples per stratum
# Not recommended, but necessary for experimentation. If you want to use it, uncomment the line in the discretize_and_split_data function
# For a more robust solution, consider using SMOTE or ADASYN
def oversample_minority_classes(df, target_category):

    class_counts = df[target_category].value_counts()
    minority_classes = class_counts[class_counts < 2].index
    
    empty_classes = [class_label for class_label in range(class_counts.index.max() + 1) if class_label not in class_counts.index]
    for empty_class in empty_classes:
        df_empty_class_resampled = resample(df[df[target_category] == empty_class],
                                            replace=True,
                                            n_samples=2,
                                            random_state=42)
        
        df_empty_class_resampled[target_category] = empty_class

        df = pd.concat([df, df_empty_class_resampled])
    
    for minority_class in minority_classes:
        df_minority = df[df[target_category] == minority_class]
        
        df_minority_upsampled = resample(df_minority, 
                                         replace=True,    
                                         n_samples=2,     
                                         random_state=42)

        df = pd.concat([df, df_minority_upsampled])
    
    return df

def discretize_and_split_data(df, target='popularity', n_bins=5, strategy='quantile', test_size=0.2, random_state=42):
    target_category = target + '_category'

    discretizer = KBinsDiscretizer(n_bins=n_bins, encode='ordinal', strategy=strategy)

    discretized_target = discretizer.fit_transform(df[target].values.reshape(-1, 1))

    unique, counts = np.unique(discretized_target, return_counts=True)
    logging.info("Distribución de categorías después de la discretización:")
    logging.info(dict(zip(unique, counts)))

    df[target_category] = discretized_target
    
    # Apply oversampling to minority classes
    #df = oversample_minority_classes(df, target_category)

    imputer = SimpleImputer(strategy='mean')
    features_imputed = imputer.fit_transform(df.drop([target, target_category], axis=1))

    # Separate features and target
    #features, target = remove_outliers(features_imputed, df[target_category])

    # Split the data into training and testing sets after ensuring at least 2 samples per stratum
    # X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=test_size, random_state=random_state, stratify=target)
    # Separate features and target
    target_series = df[target_category]
    
    return features_imputed, target_series

scoring_metrics_classification = ['balanced_accuracy', 'accuracy', 'precision_macro', 'recall_macro', 'f1_macro', 'precision_weighted', 'recall_weighted', 'f1_weighted']


In [26]:
models = {
    'RandomForestClassifier': {
        'model': RandomForestClassifier(),
        'params': {
            'model__n_estimators': randint(10, 200),
            'model__max_depth': randint(4, 100),
            'model__max_features': [None, 'sqrt', 'log2'],
            'model__class_weight': ['balanced', None],
        }
    },
    'XGBClassifier': {
        'model': XGBClassifier(),
        'params': {
            'model__n_estimators': randint(10, 200),
            'model__max_depth': randint(4, 100),
            'model__learning_rate': uniform(loc=0.03, scale=0.4),
            'model__max_leaves': randint(0, 1000),
            'model__grow_policy': ['depthwise', 'lossguide'], #0: depthwise, 1: losswise
            #'model__enable_categorical': [True],
        }
    },
    'DecisionTreeClassifier': {
        'model': DecisionTreeClassifier(),
        'params': {
            'model__max_depth': randint(4, 100),
            'model__max_features': [None, 'sqrt', 'log2'],
            'model__splitter': ['best', 'random'],
            'model__class_weight': ['balanced', None],
        }
    },
    'HistGradientBoostingClassifier': { #Tarda muy poco
        'model': HistGradientBoostingClassifier(),
        'params': {
            'model__max_iter': randint(10, 400), #More or less equivalent to n_estimators, this model have no attribute n_estimators
            'model__max_depth': randint(4, 400), 
            'model__learning_rate': uniform(loc=0.01, scale=0.4),
            'model__class_weight': ['balanced', None],
            'model__max_leaf_nodes': randint(20,500), #None = no limit, if benefit of high numbers will be set to none. 
        }
    },
    'LinearSVC': { 
        'model': LinearSVC(),
        'params': {
            'model__C': uniform(loc=0.1, scale=0.9),
            'model__max_iter': randint(500, 100000),
            'model__class_weight': ['balanced', None],
            'model__dual': ['auto'],
        }
    },
}

n_iter = 100

In [10]:
log_file_prefix = os.path.join(logs_folder, "hyperparameters_n_bins-5_uniform_")
X_train, y_train = discretize_and_split_data(df, n_bins=5, strategy='uniform')
for i in range (5):
    best_model = train_and_select_best_model(models, scalers, X_train, y_train, scoring=scoring_metrics_classification, n_iter=n_iter)


Training model: RandomForestClassifier


/home/jsolis/.local/lib/python3.11/site-packages/sklearn/preprocessing/_discretization.py:248: FutureWarning: In version 1.5 onwards, subsample=200_000 will be used by default. Set subsample explicitly to silence this warning in the mean time. Set subsample=None to disable subsampling explicitly.
  warnings.warn(
/home/jsolis/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/jsolis/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/jsolis/.local/lib/python3.1

Best score for model RandomForestClassifier with MinMaxScaler using balanced_accuracy: 0.4042692003409324
Tiempo transcurrido: 00:55:36
Training model: XGBClassifier


/home/jsolis/.local/lib/python3.11/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/home/jsolis/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/jsolis/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/jsolis/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: U

Best score for model XGBClassifier with QuantileTransformer using balanced_accuracy: 0.33133114757086946
Tiempo transcurrido: 03:06:22
Best overall model: RandomForestClassifier
Best overall score: 0.4042692003409324
Training model: RandomForestClassifier


...

Best score for model XGBClassifier with QuantileTransformer using balanced_accuracy: 0.33101975392265326
Tiempo transcurrido: 00:17:33
Best overall model: RandomForestClassifier
Best overall score: 0.39936314926605326
Training model: RandomForestClassifier


/home/jsolis/.local/lib/python3.11/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/home/jsolis/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/jsolis/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/jsolis/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: U

Best score for model RandomForestClassifier with MinMaxScaler using balanced_accuracy: 0.4006687836804521
Tiempo transcurrido: 00:55:19
Training model: XGBClassifier


/home/jsolis/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/jsolis/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/jsolis/.local/lib/python3.11/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/home/jsolis/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: U

Best score for model XGBClassifier with QuantileTransformer using balanced_accuracy: 0.33110146138002994
Tiempo transcurrido: 00:20:07
Best overall model: RandomForestClassifier
Best overall score: 0.4006687836804521


In [ ]:
log_file_prefix = os.path.join(logs_folder, "hyperparameters_n_bins-10_uniform_")
X_train, y_train = discretize_and_split_data(df, n_bins=10, strategy='uniform')
for i in range (5):
    best_model = train_and_select_best_model(models, scalers, X_train, y_train, scoring=scoring_metrics_classification, n_iter=n_iter)

Training model: RandomForestClassifier


/home/jsolis/.local/lib/python3.11/site-packages/sklearn/preprocessing/_discretization.py:248: FutureWarning: In version 1.5 onwards, subsample=200_000 will be used by default. Set subsample explicitly to silence this warning in the mean time. Set subsample=None to disable subsampling explicitly.
  warnings.warn(
/home/jsolis/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/jsolis/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/jsolis/.local/lib/python3.1

Best score for model RandomForestClassifier with MinMaxScaler using balanced_accuracy: 0.22781938249542893
Tiempo transcurrido: 01:02:19
Training model: XGBClassifier


/home/jsolis/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/jsolis/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/jsolis/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/jsolis/.local/li

Best score for model XGBClassifier with QuantileTransformer using balanced_accuracy: 0.19467592717216592
Tiempo transcurrido: 02:37:04
Best overall model: RandomForestClassifier
Best overall score: 0.22781938249542893
Training model: RandomForestClassifier


...

/home/jsolis/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/jsolis/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/jsolis/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/jsolis/.local/li

Best score for model XGBClassifier with QuantileTransformer using balanced_accuracy: 0.195131241386081
Tiempo transcurrido: 00:35:28
Best overall model: RandomForestClassifier
Best overall score: 0.23306515903326003
Training model: RandomForestClassifier


/home/jsolis/.local/lib/python3.11/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/home/jsolis/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/jsolis/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/jsolis/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: U

Best score for model RandomForestClassifier with MinMaxScaler using balanced_accuracy: 0.22449584646520523
Tiempo transcurrido: 00:54:28
Training model: XGBClassifier


/home/jsolis/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/jsolis/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/jsolis/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/jsolis/.local/li

In [ ]:
log_file_prefix = os.path.join(logs_folder, "hyperparameters_n_bins-5_quantile_")
X_train, y_train = discretize_and_split_data(df, n_bins=5, strategy='quantile')
for i in range (5):
    best_model = train_and_select_best_model(models, scalers, X_train, y_train, scoring=scoring_metrics_classification, n_iter=n_iter)

Training model: RandomForestClassifier


/home/jsolis/.local/lib/python3.11/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Best score for model RandomForestClassifier with MinMaxScaler using balanced_accuracy: 0.4001373356409765
Tiempo transcurrido: 00:57:58
Training model: XGBClassifier


/home/jsolis/.local/lib/python3.11/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


In [ ]:
log_file_prefix = os.path.join(logs_folder, "hyperparameters_n_bins-10_quantile_")
X_train, y_train = discretize_and_split_data(df, n_bins=10, strategy='quantile')
for i in range (5):
    best_model = train_and_select_best_model(models, scalers, X_train, y_train, scoring=scoring_metrics_classification, n_iter=n_iter)

LightGBM fue finalmente descartado por problemas de rendimiento

LightGBM was finally discarded due to performance issues

In [ ]:
models = { 
    'LGBMClassifier': {
        'model': LGBMClassifier(),
        'params': {
            'model__n_estimators': randint(20, 150),
            'model__max_depth': randint(-1, 50),
            'model__learning_rate': uniform(loc=0.1, scale=0.3),
            'model__num_leaves': randint(10, 60),#El algoritmo especifica que recomienda 2^max_depth > num_leaves para hacer buenas estimaciones
            'model__class_weight': ['balanced', None],
            #'model__is_unbalance': [True, False],
        }
    },
}

n_iter = 50

In [ ]:
models = { #No va bien, tarda demasiado
    'LGBMClassifier': {
        'model': LGBMClassifier(),
        'params': {
            'model__n_estimators': [20,40,60,80,100,120],
            'model__max_depth': [None],
            'model__learning_rate': [0.15],
            'model__num_leaves': [31],
        }
    },
}

In [ ]:

log_file_prefix = os.path.join(logs_folder, "hyperparameters_n_bins-5_uniform_")
X_train, y_train = discretize_and_split_data(df, n_bins=5, strategy='uniform')
best_model = train_and_select_best_model(models, scalers, X_train, y_train, scoring=scoring_metrics_classification, search_algorithm=GridSearchCV)

In [ ]:
log_file_prefix = os.path.join(logs_folder, "hyperparameters_n_bins-10_uniform_")
X_train, y_train = discretize_and_split_data(df, n_bins=10, strategy='uniform')
best_model = train_and_select_best_model(models, scalers, X_train, y_train, scoring=scoring_metrics_classification, search_algorithm=GridSearchCV)


In [ ]:

log_file_prefix = os.path.join(logs_folder, "hyperparameters_n_bins-5_quantile_")
X_train, y_train = discretize_and_split_data(df, n_bins=5, strategy='quantile')
best_model = train_and_select_best_model(models, scalers, X_train, y_train, scoring=scoring_metrics_classification, search_algorithm=GridSearchCV)


In [ ]:

log_file_prefix = os.path.join(logs_folder, "hyperparameters_n_bins-10_quantile_")
X_train, y_train = discretize_and_split_data(df, n_bins=10, strategy='quantile')
best_model = train_and_select_best_model(models, scalers, X_train, y_train, scoring=scoring_metrics_classification, search_algorithm=GridSearchCV)

In [ ]:
models = { #No va bien, tarda demasiado
    'LGBMClassifier': {
        'model': LGBMClassifier(),
        'params': {
            'model__n_estimators': [100],
            'model__max_depth': [None, 4,10,20,30,40,50],
            'model__learning_rate': [0.15],
            'model__num_leaves': [31],
            'model__class_weight': ['balanced',None],
        }
    },
}

log_file_prefix = os.path.join(logs_folder, "hyperparameters_n_bins-5_uniform_")
X_train, y_train = discretize_and_split_data(df, n_bins=5, strategy='uniform')
best_model = train_and_select_best_model(models, scalers, X_train, y_train, scoring=scoring_metrics_classification, search_algorithm=GridSearchCV)


In [ ]:
models = { #No va bien, tarda demasiado
    'LGBMClassifier': {
        'model': LGBMClassifier(),
        'params': {
            'model__n_estimators': [100],
            'model__max_depth': [None],
            'model__learning_rate': [0.1,0.15,0.2,0.25,0.3,0.4],
            'model__num_leaves': [31],
            'model__class_weight': ['balanced',None],
        }
    }, 
}

log_file_prefix = os.path.join(logs_folder, "hyperparameters_n_bins-5_uniform_")
X_train, y_train = discretize_and_split_data(df, n_bins=5, strategy='uniform')
best_model = train_and_select_best_model(models, scalers, X_train, y_train, scoring=scoring_metrics_classification, search_algorithm=GridSearchCV)


In [10]:
log_file_prefix = os.path.join(logs_folder, "optimization_final_5-bins-uniform_")
models = { 
    'DecisionTreeClassifier': {
        'model': DecisionTreeClassifier(),
        'params': {
            'model__max_features': [None],
            'model__max_depth': [3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20],
            'model__splitter': ['best'],
            'model__class_weight': ['balanced'],
        }
    },  
}

X_train, y_train = discretize_and_split_data(df, n_bins=5, strategy='uniform')
best_model = train_and_select_best_model(models, scalers, X_train, y_train, scoring=scoring_metrics_classification, search_algorithm=GridSearchCV)


Training model: DecisionTreeClassifier


/home/jsolis/.local/lib/python3.11/site-packages/sklearn/preprocessing/_discretization.py:248: FutureWarning: In version 1.5 onwards, subsample=200_000 will be used by default. Set subsample explicitly to silence this warning in the mean time. Set subsample=None to disable subsampling explicitly.
  warnings.warn(
/home/jsolis/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/jsolis/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/jsolis/.local/lib/python3.1

KeyboardInterrupt: 

In [11]:
def evaluate_model(model, X_train, y_train, scaler):

    scoring = make_scorer(balanced_accuracy_score, greater_is_better=True)
    pipeline = define_pipeline(scaler, model)
    cv_scores = cross_val_score(pipeline, X_train, y_train, cv=5, scoring=scoring)

    return cv_scores.mean()


def objective_random_forest(trial, X_train, y_train, scaler):
    model = RandomForestClassifier(
        n_estimators=trial.suggest_int('n_estimators', 23, 200),
        max_depth=trial.suggest_int('max_depth', 4, 17),
        max_features = trial.suggest_categorical('max_features', ['sqrt', 'log2']),
        class_weight = trial.suggest_categorical('class_weight',['balanced'])
    )
    return evaluate_model(model, X_train, y_train, scaler)

def objective_decision_tree(trial, X_train, y_train, scaler):
    model = DecisionTreeClassifier(
        max_depth=trial.suggest_int('max_depth', 3, 20),
        max_features=trial.suggest_categorical('max_features', [None]),
        class_weight = trial.suggest_categorical('class_weight',['balanced']),
        splitter = trial.suggest_categorical('splitter',['best']),
    )
    return evaluate_model(model, X_train, y_train, scaler)

def objective_hist_gradient_boosting(trial, X_train, y_train, scaler):
    model = HistGradientBoostingClassifier(
        max_iter=trial.suggest_int('max_iter', 10, 400),
        max_depth=trial.suggest_int('max_depth', 4, 400),
        learning_rate=trial.suggest_float('learning_rate', 0.12, 0.26),
        max_leaf_nodes=trial.suggest_int('max_leaf_nodes', 5, 100),
        class_weight = trial.suggest_categorical('class_weight',['balanced']),
    )
    return evaluate_model(model, X_train, y_train, scaler)

def objective_xgb(trial, X_train, y_train, scaler):
    model = XGBClassifier(
        n_estimators=trial.suggest_int('n_estimators', 33, 160),
        max_depth=trial.suggest_int('max_depth', 4, 100),
        learning_rate=trial.suggest_float('learning_rate', 0.01, 0.25),
        max_leaves=trial.suggest_int('max_leaves', 5, 200),
        grow_policy=trial.suggest_categorical('grow_policy', ['depthwise', 'lossguide']),
    )
    return evaluate_model(model, X_train, y_train, scaler)

def objective_linear_svc(trial, X_train, y_train, scaler):
    model = LinearSVC(
        max_iter=trial.suggest_int('max_iter', 500, 100000),
        C=trial.suggest_float('C', 0.2, 1),
        class_weight = trial.suggest_categorical('class_weight',['balanced']),
        dual=trial.suggest_categorical('dual', ['auto'])
    )
    return evaluate_model(model, X_train, y_train, scaler)


models = {
    'RandomForestClassifier': objective_random_forest,
    'XGBClassifier': objective_xgb,
    'DecisionTreeClassifier':objective_decision_tree,
    'HistGradientBoostingClassifier': objective_hist_gradient_boosting,
    'LinearSVC':objective_linear_svc,
}


In [ ]:
log_file_prefix = os.path.join(logs_folder, "optimization_final_5-bins-uniform_")
X_train, y_train = discretize_and_split_data(df, n_bins=5, strategy='uniform')
for i in range (5):
    best_model = train_and_select_best_model(models, scalers, X_train, y_train, scoring=scoring_metrics_classification, search_algorithm=optuna, n_iter=100)

/home/jsolis/.local/lib/python3.11/site-packages/sklearn/preprocessing/_discretization.py:248: FutureWarning: In version 1.5 onwards, subsample=200_000 will be used by default. Set subsample explicitly to silence this warning in the mean time. Set subsample=None to disable subsampling explicitly.
  warnings.warn(
[I 2024-05-19 23:06:49,621] A new study created in memory with name: no-name-c1a9995d-5187-485b-be20-e4008bc8ae50


Training model: HistGradientBoostingClassifier


[I 2024-05-19 23:08:11,109] Trial 10 finished with value: 0.40368423835428147 and parameters: {'max_iter': 69, 'max_depth': 4, 'learning_rate': 0.22743471058661285, 'max_leaf_nodes': 68, 'class_weight': 'balanced'}. Best is trial 10 with value: 0.40368423835428147.
[I 2024-05-19 23:08:19,933] Trial 19 finished with value: 0.3973462400263666 and parameters: {'max_iter': 137, 'max_depth': 30, 'learning_rate': 0.23403447475253894, 'max_leaf_nodes': 13, 'class_weight': 'balanced'}. Best is trial 10 with value: 0.40368423835428147.
[I 2024-05-19 23:08:29,355] Trial 17 finished with value: 0.40226877754173856 and parameters: {'max_iter': 54, 'max_depth': 328, 'learning_rate': 0.19111918398013222, 'max_leaf_nodes': 16, 'class_weight': 'balanced'}. Best is trial 10 with value: 0.40368423835428147.
[I 2024-05-19 23:08:30,158] Trial 7 finished with value: 0.4000812571655449 and parameters: {'max_iter': 363, 'max_depth': 213, 'learning_rate': 0.18745905191522802, 'max_leaf_nodes': 14, 'class_weig

Best score for model HistGradientBoostingClassifier with RobustScaler using balanced_accuracy: 0.41304014054239485
Tiempo transcurrido: 00:13:20
Training model: LinearSVC


[I 2024-05-19 23:21:04,123] Trial 17 finished with value: 0.3862184145863151 and parameters: {'max_iter': 94753, 'C': 0.6669394667308384, 'class_weight': 'balanced', 'dual': 'auto'}. Best is trial 17 with value: 0.3862184145863151.
[I 2024-05-19 23:21:25,332] Trial 11 finished with value: 0.3867257725471135 and parameters: {'max_iter': 97851, 'C': 0.7459206097746351, 'class_weight': 'balanced', 'dual': 'auto'}. Best is trial 11 with value: 0.3867257725471135.
[I 2024-05-19 23:21:42,847] Trial 0 finished with value: 0.3869834273945605 and parameters: {'max_iter': 16217, 'C': 0.3049849450265762, 'class_weight': 'balanced', 'dual': 'auto'}. Best is trial 0 with value: 0.3869834273945605.
[I 2024-05-19 23:21:47,029] Trial 4 finished with value: 0.3872489304735434 and parameters: {'max_iter': 31922, 'C': 0.8456298751339439, 'class_weight': 'balanced', 'dual': 'auto'}. Best is trial 4 with value: 0.3872489304735434.
[I 2024-05-19 23:21:52,297] Trial 19 finished with value: 0.3875199809081663

...

[I 2024-05-20 00:25:44,852] Trial 407 finished with value: 0.4103952088212807 and parameters: {'max_iter': 334, 'max_depth': 193, 'learning_rate': 0.17570594521363328, 'max_leaf_nodes': 6, 'class_weight': 'balanced'}. Best is trial 158 with value: 0.419645227661268.
[I 2024-05-20 00:25:53,028] Trial 418 finished with value: 0.40750117244596373 and parameters: {'max_iter': 146, 'max_depth': 193, 'learning_rate': 0.17633016027024312, 'max_leaf_nodes': 6, 'class_weight': 'balanced'}. Best is trial 158 with value: 0.419645227661268.
[I 2024-05-20 00:25:56,627] Trial 406 finished with value: 0.409588578159096 and parameters: {'max_iter': 152, 'max_depth': 184, 'learning_rate': 0.17576509505943966, 'max_leaf_nodes': 6, 'class_weight': 'balanced'}. Best is trial 158 with value: 0.419645227661268.
[I 2024-05-20 00:25:59,681] Trial 415 finished with value: 0.4132808177500086 and parameters: {'max_iter': 148, 'max_depth': 191, 'learning_rate': 0.17586148315710542, 'max_leaf_nodes': 6, 'class_wei

Best score for model HistGradientBoostingClassifier with RobustScaler using balanced_accuracy: 0.419645227661268
Tiempo transcurrido: 00:11:35
Training model: LinearSVC


[I 2024-05-20 00:36:49,207] Trial 408 finished with value: 0.38680541312285355 and parameters: {'max_iter': 33732, 'C': 0.3536538739588604, 'class_weight': 'balanced', 'dual': 'auto'}. Best is trial 262 with value: 0.38841114870820254.
[I 2024-05-20 00:36:50,811] Trial 415 finished with value: 0.386758765060086 and parameters: {'max_iter': 35240, 'C': 0.6740404085356015, 'class_weight': 'balanced', 'dual': 'auto'}. Best is trial 262 with value: 0.38841114870820254.
[I 2024-05-20 00:36:52,757] Trial 406 finished with value: 0.38688458130546655 and parameters: {'max_iter': 32555, 'C': 0.35095724845805454, 'class_weight': 'balanced', 'dual': 'auto'}. Best is trial 262 with value: 0.38841114870820254.
[I 2024-05-20 00:36:55,147] Trial 418 finished with value: 0.3861251479788555 and parameters: {'max_iter': 34763, 'C': 0.4254809609032838, 'class_weight': 'balanced', 'dual': 'auto'}. Best is trial 262 with value: 0.38841114870820254.
[I 2024-05-20 00:36:55,609] Trial 400 finished with value:

Best score for model LinearSVC with QuantileTransformer using balanced_accuracy: 0.38841114870820254
Tiempo transcurrido: 00:07:27
Best overall model: HistGradientBoostingClassifier
Best overall score: 0.419645227661268
Training model: HistGradientBoostingClassifier


[I 2024-05-20 00:44:52,729] Trial 517 finished with value: 0.412980706175542 and parameters: {'max_iter': 138, 'max_depth': 28, 'learning_rate': 0.1617257998792118, 'max_leaf_nodes': 7, 'class_weight': 'balanced'}. Best is trial 158 with value: 0.419645227661268.
[I 2024-05-20 00:45:01,387] Trial 514 finished with value: 0.40831464477107593 and parameters: {'max_iter': 140, 'max_depth': 27, 'learning_rate': 0.16052144245885802, 'max_leaf_nodes': 7, 'class_weight': 'balanced'}. Best is trial 158 with value: 0.419645227661268.
[I 2024-05-20 00:45:05,995] Trial 503 finished with value: 0.40804701475770067 and parameters: {'max_iter': 134, 'max_depth': 166, 'learning_rate': 0.18448154446469114, 'max_leaf_nodes': 7, 'class_weight': 'balanced'}. Best is trial 158 with value: 0.419645227661268.
[I 2024-05-20 00:45:10,444] Trial 519 finished with value: 0.412091193231337 and parameters: {'max_iter': 139, 'max_depth': 201, 'learning_rate': 0.16265146374962014, 'max_leaf_nodes': 7, 'class_weight

In [ ]:
log_file_prefix = os.path.join(logs_folder, "optimization_final_10-bins-uniform_")

def evaluate_model(model, X_train, y_train, scaler):

    scoring = make_scorer(balanced_accuracy_score, greater_is_better=True)
    pipeline = define_pipeline(scaler, model)
    cv_scores = cross_val_score(pipeline, X_train, y_train, cv=5, scoring=scoring)

    return cv_scores.mean()


def objective_random_forest(trial, X_train, y_train, scaler):
    model = RandomForestClassifier(
        n_estimators=trial.suggest_int('n_estimators', 50, 200),
        max_depth=trial.suggest_int('max_depth', 3, 18),
        max_features = trial.suggest_categorical('max_features', ['sqrt', 'log2']),
        class_weight = trial.suggest_categorical('class_weight',['balanced'])
    )
    return evaluate_model(model, X_train, y_train, scaler)

def objective_decision_tree(trial, X_train, y_train, scaler):
    model = DecisionTreeClassifier(
        max_depth=trial.suggest_int('max_depth', 3, 16),
        max_features=trial.suggest_categorical('max_features', ['log2','sqrt', None]),
        class_weight = trial.suggest_categorical('class_weight',['balanced']),
        splitter = trial.suggest_categorical('splitter',['best']),
    )
    return evaluate_model(model, X_train, y_train, scaler)

def objective_hist_gradient_boosting(trial, X_train, y_train, scaler):
    model = HistGradientBoostingClassifier(
        max_iter=trial.suggest_int('max_iter', 10, 400),
        max_depth=trial.suggest_int('max_depth', 4, 400),
        learning_rate=trial.suggest_float('learning_rate', 0.01, 0.15),
        max_leaf_nodes=trial.suggest_int('max_leaf_nodes', 10, 120),
        class_weight = trial.suggest_categorical('class_weight',['balanced']),
    )
    return evaluate_model(model, X_train, y_train, scaler)

def objective_xgb(trial, X_train, y_train, scaler):
    model = XGBClassifier(
        n_estimators=trial.suggest_int('n_estimators', 20, 100),
        max_depth=trial.suggest_int('max_depth', 4, 100),
        learning_rate=trial.suggest_float('learning_rate', 0.01, 0.2),
        max_leaves=trial.suggest_int('max_leaves', 10, 200),
        grow_policy=trial.suggest_categorical('grow_policy', ['depthwise', 'lossguide']),
    )
    return evaluate_model(model, X_train, y_train, scaler)

def objective_linear_svc(trial, X_train, y_train, scaler):
    model = LinearSVC(
        max_iter=trial.suggest_int('max_iter', 500, 100000),
        C=trial.suggest_float('C', 0.1, 0.8),
        class_weight = trial.suggest_categorical('class_weight',['balanced']),
        dual=trial.suggest_categorical('dual', ['auto'])
    )
    return evaluate_model(model, X_train, y_train, scaler)


models = {
    'DecisionTreeClassifier':objective_decision_tree,
    'RandomForestClassifier': objective_random_forest,
    'XGBClassifier': objective_xgb,
    'HistGradientBoostingClassifier': objective_hist_gradient_boosting,
    'LinearSVC':objective_linear_svc,
}

X_train, y_train = discretize_and_split_data(df, n_bins=10, strategy='uniform')
for i in range (5):
    best_model = train_and_select_best_model(models, scalers, X_train, y_train, scoring=scoring_metrics_classification, search_algorithm=optuna, n_iter=100)

/home/jsolis/.local/lib/python3.11/site-packages/sklearn/preprocessing/_discretization.py:248: FutureWarning: In version 1.5 onwards, subsample=200_000 will be used by default. Set subsample explicitly to silence this warning in the mean time. Set subsample=None to disable subsampling explicitly.
  warnings.warn(
[I 2024-05-20 14:33:22,189] A new study created in memory with name: no-name-3674cfb8-8e36-40d8-bd19-0da465b47bcb


Training model: DecisionTreeClassifier


[I 2024-05-20 14:33:30,437] Trial 9 finished with value: 0.18014495567605776 and parameters: {'max_depth': 5, 'max_features': 'sqrt', 'class_weight': 'balanced', 'splitter': 'best'}. Best is trial 9 with value: 0.18014495567605776.
[I 2024-05-20 14:33:31,045] Trial 11 finished with value: 0.19706967954918175 and parameters: {'max_depth': 6, 'max_features': 'sqrt', 'class_weight': 'balanced', 'splitter': 'best'}. Best is trial 11 with value: 0.19706967954918175.
[I 2024-05-20 14:33:32,762] Trial 13 finished with value: 0.1986724852395804 and parameters: {'max_depth': 10, 'max_features': 'sqrt', 'class_weight': 'balanced', 'splitter': 'best'}. Best is trial 13 with value: 0.1986724852395804.
[I 2024-05-20 14:33:33,137] Trial 10 finished with value: 0.19369059998883853 and parameters: {'max_depth': 11, 'max_features': 'sqrt', 'class_weight': 'balanced', 'splitter': 'best'}. Best is trial 13 with value: 0.1986724852395804.
[I 2024-05-20 14:33:33,261] Trial 18 finished with value: 0.1942184

Best score for model DecisionTreeClassifier with MinMaxScaler using balanced_accuracy: 0.21598456169121136
Tiempo transcurrido: 00:00:52
Training model: RandomForestClassifier


[I 2024-05-20 14:36:24,573] Trial 6 finished with value: 0.22401074314464126 and parameters: {'n_estimators': 58, 'max_depth': 8, 'max_features': 'sqrt', 'class_weight': 'balanced'}. Best is trial 6 with value: 0.22401074314464126.
[I 2024-05-20 14:36:26,377] Trial 1 finished with value: 0.22962193654596802 and parameters: {'n_estimators': 78, 'max_depth': 6, 'max_features': 'log2', 'class_weight': 'balanced'}. Best is trial 1 with value: 0.22962193654596802.
[I 2024-05-20 14:36:32,558] Trial 2 finished with value: 0.22954574703005637 and parameters: {'n_estimators': 142, 'max_depth': 3, 'max_features': 'sqrt', 'class_weight': 'balanced'}. Best is trial 1 with value: 0.22962193654596802.
[I 2024-05-20 14:36:36,451] Trial 7 finished with value: 0.22423743262439708 and parameters: {'n_estimators': 63, 'max_depth': 8, 'max_features': 'sqrt', 'class_weight': 'balanced'}. Best is trial 1 with value: 0.22962193654596802.
[I 2024-05-20 14:36:39,821] Trial 8 finished with value: 0.222497829895

...

[I 2024-05-20 20:27:18,094] Trial 107 finished with value: 0.19268659556689338 and parameters: {'max_depth': 11, 'max_features': 'log2', 'class_weight': 'balanced', 'splitter': 'best'}. Best is trial 23 with value: 0.21598456169121136.
[I 2024-05-20 20:27:19,594] Trial 111 finished with value: 0.20129706685810383 and parameters: {'max_depth': 11, 'max_features': 'log2', 'class_weight': 'balanced', 'splitter': 'best'}. Best is trial 23 with value: 0.21598456169121136.
[I 2024-05-20 20:27:20,086] Trial 117 finished with value: 0.19410221506709815 and parameters: {'max_depth': 11, 'max_features': 'log2', 'class_weight': 'balanced', 'splitter': 'best'}. Best is trial 23 with value: 0.21598456169121136.
[I 2024-05-20 20:27:20,693] Trial 112 finished with value: 0.19487724763174802 and parameters: {'max_depth': 11, 'max_features': 'log2', 'class_weight': 'balanced', 'splitter': 'best'}. Best is trial 23 with value: 0.21598456169121136.
[I 2024-05-20 20:27:20,953] Trial 110 finished with valu

Best score for model DecisionTreeClassifier with MinMaxScaler using balanced_accuracy: 0.2223015133508636
Tiempo transcurrido: 00:00:50
Training model: RandomForestClassifier


[I 2024-05-20 20:32:46,467] Trial 112 finished with value: 0.22886502904003972 and parameters: {'n_estimators': 165, 'max_depth': 5, 'max_features': 'sqrt', 'class_weight': 'balanced'}. Best is trial 21 with value: 0.24053227852360495.
[I 2024-05-20 20:32:48,518] Trial 104 finished with value: 0.23149200161552214 and parameters: {'n_estimators': 165, 'max_depth': 5, 'max_features': 'sqrt', 'class_weight': 'balanced'}. Best is trial 21 with value: 0.24053227852360495.
[I 2024-05-20 20:32:50,446] Trial 113 finished with value: 0.23866715288635296 and parameters: {'n_estimators': 165, 'max_depth': 5, 'max_features': 'sqrt', 'class_weight': 'balanced'}. Best is trial 21 with value: 0.24053227852360495.
[I 2024-05-20 20:32:51,382] Trial 105 finished with value: 0.22923706627698795 and parameters: {'n_estimators': 165, 'max_depth': 5, 'max_features': 'sqrt', 'class_weight': 'balanced'}. Best is trial 21 with value: 0.24053227852360495.
[I 2024-05-20 20:32:52,307] Trial 118 finished with valu

Best score for model RandomForestClassifier with MinMaxScaler using balanced_accuracy: 0.24208589193511196
Tiempo transcurrido: 00:24:01
Training model: XGBClassifier


[I 2024-05-20 20:57:44,475] Trial 119 finished with value: 0.1887849498791323 and parameters: {'n_estimators': 75, 'max_depth': 93, 'learning_rate': 0.09851633859398659, 'max_leaves': 171, 'grow_policy': 'depthwise'}. Best is trial 75 with value: 0.19489262653417888.
[I 2024-05-20 20:59:52,106] Trial 102 finished with value: 0.19160590067733585 and parameters: {'n_estimators': 76, 'max_depth': 93, 'learning_rate': 0.15142321743375875, 'max_leaves': 169, 'grow_policy': 'depthwise'}. Best is trial 75 with value: 0.19489262653417888.
[I 2024-05-20 20:59:56,527] Trial 103 finished with value: 0.18873824529393723 and parameters: {'n_estimators': 76, 'max_depth': 92, 'learning_rate': 0.09632893499608906, 'max_leaves': 134, 'grow_policy': 'depthwise'}. Best is trial 75 with value: 0.19489262653417888.
[I 2024-05-20 21:01:00,996] Trial 109 finished with value: 0.18879746594403102 and parameters: {'n_estimators': 83, 'max_depth': 93, 'learning_rate': 0.09529243026702205, 'max_leaves': 135, 'gro

In [ ]:
#log_file_prefix = os.path.join(logs_folder, "optimization_final_5-bins-quantile_")
#X_train, y_train = discretize_and_split_data(df, n_bins=5, strategy='quantile')
#best_model = train_and_select_best_model(models, scalers, X_train, y_train, scoring=scoring_metrics_classification, search_algorithm=optuna, n_iter=1)

[I 2024-04-11 23:29:50,751] A new study created in memory with name: no-name-6b1931ac-eabd-41b8-8477-002f5968ee20


Training model: RandomForestClassifier


[I 2024-04-11 23:29:52,488] Trial 0 finished with value: -2.4178887212839766 and parameters: {'n_estimators': 94, 'max_depth': 5}. Best is trial 0 with value: -2.4178887212839766.
[I 2024-04-11 23:29:52,490] A new study created in memory with name: no-name-4129b4dc-068e-4c51-b870-080e78797c6a


Best score for model RandomForestClassifier with MaxAbsScaler using accuracy: -2.4178887212839766
Tiempo transcurrido: 00:00:01
Training model: DecisionTreeClassifier


[I 2024-04-11 23:29:52,726] Trial 0 finished with value: -2.873664589797793 and parameters: {'max_depth': 19, 'max_features': 'sqrt'}. Best is trial 0 with value: -2.873664589797793.
[I 2024-04-11 23:29:52,728] A new study created in memory with name: no-name-7ac2fbfe-c9e1-43ff-adee-5e2b33fc1a73


Best score for model DecisionTreeClassifier with QuantileTransformer using accuracy: -2.873664589797793
Tiempo transcurrido: 00:00:00
Training model: LGBMClassifier
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000408 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2565
[LightGBM] [Info] Number of data points in the train set: 10135, number of used features: 13
[LightGBM] [Info] Start training from score -1.703516
[LightGBM] [Info] Start training from score -1.574534
[LightGBM] [Info] Start training from score -1.552855
[LightGBM] [Info] Start training from score -1.601575
[LightGBM] [Info] Start training from score -1.621349
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positiv

[I 2024-04-11 23:29:53,815] Trial 0 finished with value: -2.3314593311915033 and parameters: {'n_estimators': 15, 'max_depth': 49, 'learning_rate': 0.3142446116245381, 'num_leaves': 466}. Best is trial 0 with value: -2.3314593311915033.
[I 2024-04-11 23:29:53,818] A new study created in memory with name: no-name-22f712f8-b186-4625-99ab-fe16a3cda8a6


Best score for model LGBMClassifier with MaxAbsScaler using accuracy: -2.3314593311915033
Tiempo transcurrido: 00:00:01
Training model: XGBClassifier


[I 2024-04-11 23:29:54,155] Trial 0 finished with value: -2.27778584706307 and parameters: {'n_estimators': 17, 'max_depth': 44, 'learning_rate': 0.2355335326594291, 'max_leaves': 15, 'grow_policy': 'lossguide'}. Best is trial 0 with value: -2.27778584706307.


Best score for model XGBClassifier with RobustScaler using accuracy: -2.27778584706307
Tiempo transcurrido: 00:00:00
Best overall model: XGBClassifier
Best overall score: -2.27778584706307


In [ ]:
#log_file_prefix = os.path.join(logs_folder, "optimization_final_10-bins-quantile_")
#X_train, y_train = discretize_and_split_data(df, n_bins=10, strategy='quantile')
#best_model = train_and_select_best_model(models, scalers, X_train, y_train, scoring=scoring_metrics_classification, search_algorithm=optuna, n_iter=1)

[I 2024-04-11 23:30:10,477] A new study created in memory with name: no-name-d1260222-b0ad-4674-8d71-d8d36fec971e


Training model: RandomForestClassifier


[I 2024-04-11 23:30:14,014] Trial 0 finished with value: -9.309657080089666 and parameters: {'n_estimators': 55, 'max_depth': 40}. Best is trial 0 with value: -9.309657080089666.
[I 2024-04-11 23:30:14,017] A new study created in memory with name: no-name-735f6a2b-f283-4918-b416-ec6ca8771910


Best score for model RandomForestClassifier with MaxAbsScaler using accuracy: -9.309657080089666
Tiempo transcurrido: 00:00:03
Training model: DecisionTreeClassifier


[I 2024-04-11 23:30:14,536] Trial 0 finished with value: -10.674959013265664 and parameters: {'max_depth': 17, 'max_features': None}. Best is trial 0 with value: -10.674959013265664.
[I 2024-04-11 23:30:14,538] A new study created in memory with name: no-name-2bc10626-2a65-4f62-bfc9-25c83d78a6b7


Best score for model DecisionTreeClassifier with QuantileTransformer using accuracy: -10.674959013265664
Tiempo transcurrido: 00:00:00
Training model: LGBMClassifier
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000462 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2566
[LightGBM] [Info] Number of data points in the train set: 10135, number of used features: 13
[LightGBM] [Info] Start training from score -2.456407
[LightGBM] [Info] Start training from score -2.340287
[LightGBM] [Info] Start training from score -2.270066
[LightGBM] [Info] Start training from score -2.266253
[LightGBM] [Info] Start training from score -2.292278
[LightGBM] [Info] Start training from score -2.201774
[LightGBM] [Info] Start training from score -2.326045
[LightGBM] [Info] Start training from score -2.264352
[LightGBM] [Info] Start training from score -2.345424
[LightGBM] [Info] Start training from score -2.283528
[Ligh

[I 2024-04-11 23:30:17,583] Trial 0 finished with value: -9.088395385655918 and parameters: {'n_estimators': 36, 'max_depth': 16, 'learning_rate': 0.13457709066254936, 'num_leaves': 284}. Best is trial 0 with value: -9.088395385655918.
[I 2024-04-11 23:30:17,586] A new study created in memory with name: no-name-0005fd3a-fd15-4113-b51c-a64b28b493d0


Best score for model LGBMClassifier with MaxAbsScaler using accuracy: -9.088395385655918
Tiempo transcurrido: 00:00:03
Training model: XGBClassifier


[I 2024-04-11 23:30:26,326] Trial 0 finished with value: -9.354989819753767 and parameters: {'n_estimators': 34, 'max_depth': 44, 'learning_rate': 0.16307423991101344, 'max_leaves': 436, 'grow_policy': 'depthwise'}. Best is trial 0 with value: -9.354989819753767.


Best score for model XGBClassifier with RobustScaler using accuracy: -9.354989819753767
Tiempo transcurrido: 00:00:08
Best overall model: LGBMClassifier
Best overall score: -9.088395385655918


Evaluación Train-test

In [20]:
import pandas as pd
import numpy as np
import os
import joblib
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.ensemble import GradientBoostingRegressor, HistGradientBoostingRegressor, RandomForestRegressor, RandomForestClassifier
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor, XGBClassifier
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import LinearSVR, LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, explained_variance_score, mean_absolute_percentage_error
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, balanced_accuracy_score
from sklearn.preprocessing import StandardScaler, RobustScaler, PowerTransformer, Normalizer, MinMaxScaler, MaxAbsScaler, QuantileTransformer

df = pd.read_csv('docs/600k_tracks_anonimous.csv')

features = df.drop('popularity', axis=1)
target = df['popularity']

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.33, random_state=42)#stratify=target

imputer = SimpleImputer(strategy='mean')
X_train = imputer.fit_transform(X_train)
X_test = imputer.transform(X_test)

ruta_carpeta_estudios = "jobs"

def load_scaler_config(scaler_config_file):
    scaler_config = {}
    if os.path.exists(scaler_config_file):
        with open(scaler_config_file, 'r') as csvfile:
            reader = csv.reader(csvfile)
            for row in reader:
                if len(row) >= 2:
                    scaler_config[row[0]] = row[1]
                else:
                    print("Error: Hay lineas en blanco:", row)
    return scaler_config


scaler_config_file = 'logs/scaler_config.csv'
scaler_config = load_scaler_config(scaler_config_file)

metricas_regresion = pd.DataFrame(columns=['Modelo', 'MSE', 'RMSE', 'MAE', 'MAPE', 'R2', 'Explained_Variance'])
metricas_clasificacion = pd.DataFrame(columns=['Modelo', 'Balanced_Accuracy', 'Accuracy', 'Precision_Macro', 'Recall_Macro', 'F1_Macro', 'Precision_Weighted', 'Recall_Weighted', 'F1_Weighted'])

def evaluar_modelo_regresion(nombre_modelo, modelo, X_test, y_test, scaler):
    X_test_scaled = scaler.transform(X_test)
    y_pred = modelo.predict(X_test_scaled)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    explained_variance = explained_variance_score(y_test, y_pred)
    mape = mean_absolute_percentage_error(y_test, y_pred)
    
    if r2 < 0:
        r2 = 0
    if explained_variance < 0:
        explained_variance = 0
    
    return {'Modelo': nombre_modelo, 'MSE': mse, 'RMSE': rmse, 'MAE': mae, 'R2': r2, 'Explained_Variance': explained_variance, 'MAPE': mape}


def evaluar_modelo_clasificacion(nombre_modelo, modelo, X_test, y_test, scaler):
    X_test_scaled = scaler.transform(X_test)
    y_pred = modelo.predict(X_test_scaled)
    balanced_acc = balanced_accuracy_score(y_test, y_pred)
    accuracy = accuracy_score(y_test, y_pred)
    precision_macro = precision_score(y_test, y_pred, average='macro')
    recall_macro = recall_score(y_test, y_pred, average='macro')
    f1_macro = f1_score(y_test, y_pred, average='macro')
    precision_weighted = precision_score(y_test, y_pred, average='weighted')
    recall_weighted = recall_score(y_test, y_pred, average='weighted')
    f1_weighted = f1_score(y_test, y_pred, average='weighted')
    
    
    return {
        'Modelo': nombre_modelo, 
        'Balanced_Accuracy': balanced_acc, 
        'Accuracy': accuracy, 
        'Precision_Macro': precision_macro, 
        'Recall_Macro': recall_macro, 
        'F1_Macro': f1_macro, 
        'Precision_Weighted': precision_weighted, 
        'Recall_Weighted': recall_weighted, 
        'F1_Weighted': f1_weighted
    }


for archivo in os.listdir(ruta_carpeta_estudios):
    if archivo.endswith(".pkl"):  
        print(archivo)
        estudio = joblib.load(f"{ruta_carpeta_estudios}/{archivo}")

        mejores_hiperparametros = estudio.best_params
        print(mejores_hiperparametros)
        
        algorith = archivo.replace(".pkl", "")
        modelo = None
        scaler = None
        if algorith in scaler_config:
            scaler_name = scaler_config[algorith]
            scaler = eval(scaler_name)()
            X_train_scaled = scaler.fit_transform(X_train)
        else:
            scaler = StandardScaler()
            X_train_scaled = scaler.fit_transform(X_train)
        print(scaler)
        
        if algorith == "LinearRegression":
            modelo = LinearRegression(**mejores_hiperparametros)
        elif algorith == "Lasso":
            modelo = Lasso(**mejores_hiperparametros)
        elif algorith == "Ridge":
            modelo = Ridge(**mejores_hiperparametros)
        elif algorith == "ElasticNet":
            modelo = ElasticNet(**mejores_hiperparametros)
        elif algorith == "GradientBoostingRegressor":
            modelo = GradientBoostingRegressor(**mejores_hiperparametros)
        elif algorith == "HistGradientBoostingRegressor":
            modelo = HistGradientBoostingRegressor(**mejores_hiperparametros)
        elif algorith == "LGBMRegressor":
            modelo = LGBMRegressor(n_jobs = 20, **mejores_hiperparametros)
        elif algorith == "XGBRegressor":
            modelo = XGBRegressor(n_jobs = 20, **mejores_hiperparametros)
        elif algorith == "RandomForestRegressor":
            modelo = RandomForestRegressor(n_jobs = 20, **mejores_hiperparametros)
        elif algorith == "KNeighborsRegressor":
            modelo = KNeighborsRegressor(n_jobs = 20, **mejores_hiperparametros)
        elif algorith == "LinearSVR":
            modelo = LinearSVR(**mejores_hiperparametros)
        elif algorith == "XGBClassifier":
            modelo = XGBClassifier(n_jobs = 20, **mejores_hiperparametros)
        elif algorith == "RandomForestClassifier":
            modelo = RandomForestClassifier(n_jobs = 20, **mejores_hiperparametros)
        elif algorith == "HistGradientBoostingClassifier":
            modelo = HistGradientBoostingClassifier(**mejores_hiperparametros)
        elif algorith == "DecisionTreeClassifier":
            modelo = DecisionTreeClassifier(**mejores_hiperparametros)
        elif algorith == "LinearSVC":
            modelo = LinearSVC(**mejores_hiperparametros)
        if modelo is None:
            continue
        
        print('Entrenando modelo')
        modelo.fit(X_train_scaled, y_train)
        print('Modelo entrenado')

        if 'Classifier' in algorith or algorith == "LinearSVC":
            metricas = evaluar_modelo_clasificacion(algorith, modelo, X_test, y_test, scaler)
            metricas_clasificacion = pd.concat([metricas_clasificacion, pd.DataFrame([metricas])], ignore_index=True)
        else:
            metricas = evaluar_modelo_regresion(algorith, modelo, X_test, y_test, scaler)
            metricas_regresion = pd.concat([metricas_regresion, pd.DataFrame([metricas])], ignore_index=True)

        if not os.path.exists('train-test_models'):
            os.makedirs('train-test_models')
        ruta_guardado_modelo = f"train-test_models/{archivo}" 
        joblib.dump(modelo, ruta_guardado_modelo)
        print(f"Modelo guardado en {ruta_guardado_modelo}")


metricas_regresion.to_csv('train-test_models/resultados_metricas_regresion.csv', index=False)
metricas_clasificacion.to_csv('train-test_models/resultados_metricas_clasificacion.csv', index=False)
print('Métricas de regresión guardadas en resultados_metricas_regresion.csv')
print('Métricas de clasificación guardadas en resultados_metricas_clasificacion.csv')

LinearSVR.pkl
{'dual': 'auto', 'C': 0.14290067275948565, 'max_iter': 95591}
StandardScaler()
Entrenando modelo
Modelo entrenado
Modelo guardado en train-test_models/LinearSVR.pkl
KNeighborsRegressor.pkl
{'n_neighbors': 69, 'weights': 'distance', 'algorithm': 'brute', 'p': 1}
StandardScaler()
Entrenando modelo
Modelo entrenado


/tmp/ipykernel_8167/3942407325.py:168: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  metricas_regresion = pd.concat([metricas_regresion, pd.DataFrame([metricas])], ignore_index=True)


Modelo guardado en train-test_models/KNeighborsRegressor.pkl
Ridge.pkl
{'alpha': 0.02613865143852783, 'solver': 'cholesky', 'max_iter': 14801}
StandardScaler()
Entrenando modelo
Modelo entrenado
Modelo guardado en train-test_models/Ridge.pkl
LGBMRegressor.pkl
{'n_estimators': 118, 'max_depth': 10, 'learning_rate': 0.15859769003466326, 'num_leaves': 57}
StandardScaler()
Entrenando modelo
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.008979 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2573
[LightGBM] [Info] Number of data points in the train set: 393070, number of used features: 14
[LightGBM] [Info] Start training from score 27.549012
Modelo entrenado
Modelo guardado en train-test_models/LGBMRegressor.pkl
RandomForestRegressor.pkl
{'n_estimators': 199, 'max_depth': 32, 'max_features': 'sqrt'}
StandardScaler()
Entrenando modelo
Modelo

In [21]:
import pandas as pd
import numpy as np
import os
import joblib
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.ensemble import GradientBoostingRegressor, HistGradientBoostingRegressor, RandomForestRegressor, RandomForestClassifier
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor, XGBClassifier
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import LinearSVR, LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, explained_variance_score, mean_absolute_percentage_error
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, balanced_accuracy_score
from sklearn.preprocessing import StandardScaler, RobustScaler, PowerTransformer, Normalizer, MinMaxScaler, MaxAbsScaler, QuantileTransformer

# Leer el conjunto de datos
df = pd.read_csv('docs/600k_tracks_anonimous.csv')

def discretize_and_split_data_train_test(df, target='popularity', n_bins=5, strategy='quantile', test_size=0.33, random_state=42):
    target_category = target + '_category'
    
    discretizer = KBinsDiscretizer(n_bins=n_bins, encode='ordinal', strategy=strategy)
    
    discretized_target = discretizer.fit_transform(df[target].values.reshape(-1, 1))
    
    unique, counts = np.unique(discretized_target, return_counts=True)
    logging.info("Distribución de categorías después de la discretización:")
    logging.info(dict(zip(unique, counts)))
    
    df[target_category] = discretized_target
    
    imputer = SimpleImputer(strategy='mean')
    features_imputed = imputer.fit_transform(df.drop([target, target_category], axis=1))
    
    features = features_imputed
    target = df[target_category]
    
    X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=test_size, random_state=random_state, stratify=target)
    
    return X_train, X_test, y_train, y_test


ruta_carpeta_estudios = "jobs/5-bins-uniform"

X_train, X_test, y_train, y_test = discretize_and_split_data_train_test(df, n_bins=5, strategy='uniform')

def load_scaler_config(scaler_config_file):
    scaler_config = {}
    if os.path.exists(scaler_config_file):
        with open(scaler_config_file, 'r') as csvfile:
            reader = csv.reader(csvfile)
            for row in reader:
                if len(row) >= 2:
                    scaler_config[row[0]] = row[1]
                else:
                    print("Error: Hay lineas en blanco:", row)
    return scaler_config


scaler_config_file = 'logs/scaler_config.csv'
scaler_config = load_scaler_config(scaler_config_file)


metricas_regresion = pd.DataFrame(columns=['Modelo', 'MSE', 'RMSE', 'MAE', 'MAPE', 'R2', 'Explained_Variance'])
metricas_clasificacion = pd.DataFrame(columns=['Modelo', 'Balanced_Accuracy', 'Accuracy', 'Precision_Macro', 'Recall_Macro', 'F1_Macro', 'Precision_Weighted', 'Recall_Weighted', 'F1_Weighted'])

def evaluar_modelo_regresion(nombre_modelo, modelo, X_test, y_test, scaler):
    X_test_scaled = scaler.transform(X_test)
    y_pred = modelo.predict(X_test_scaled)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    explained_variance = explained_variance_score(y_test, y_pred)
    mape = mean_absolute_percentage_error(y_test, y_pred)
    
    if r2 < 0:
        r2 = 0
    if explained_variance < 0:
        explained_variance = 0
    
    if not np.isfinite(mse):
        mse = "NaN"
    if not np.isfinite(rmse):
        rmse = "NaN"
    if not np.isfinite(mae):
        mae = "NaN"
    if not np.isfinite(mape):
        mape = "NaN"
    
    return {'Modelo': nombre_modelo, 'MSE': mse, 'RMSE': rmse, 'MAE': mae, 'R2': r2, 'Explained_Variance': explained_variance, 'MAPE': mape}


def evaluar_modelo_clasificacion(nombre_modelo, modelo, X_test, y_test, scaler):
    X_test_scaled = scaler.transform(X_test)
    y_pred = modelo.predict(X_test_scaled)
    balanced_acc = balanced_accuracy_score(y_test, y_pred)
    accuracy = accuracy_score(y_test, y_pred)
    precision_macro = precision_score(y_test, y_pred, average='macro')
    recall_macro = recall_score(y_test, y_pred, average='macro')
    f1_macro = f1_score(y_test, y_pred, average='macro')
    precision_weighted = precision_score(y_test, y_pred, average='weighted')
    recall_weighted = recall_score(y_test, y_pred, average='weighted')
    f1_weighted = f1_score(y_test, y_pred, average='weighted')
    

    if not np.isfinite(balanced_acc):
        balanced_acc = 0
    if not np.isfinite(accuracy):
        accuracy = 0
    if not np.isfinite(precision_macro):
        precision_macro = 0
    if not np.isfinite(recall_macro):
        recall_macro = 0
    if not np.isfinite(f1_macro):
        f1_macro = 0
    if not np.isfinite(precision_weighted):
        precision_weighted = 0
    if not np.isfinite(recall_weighted):
        recall_weighted = 0
    if not np.isfinite(f1_weighted):
        f1_weighted = 0
    
    return {
        'Modelo': nombre_modelo, 
        'Balanced_Accuracy': balanced_acc, 
        'Accuracy': accuracy, 
        'Precision_Macro': precision_macro, 
        'Recall_Macro': recall_macro, 
        'F1_Macro': f1_macro, 
        'Precision_Weighted': precision_weighted, 
        'Recall_Weighted': recall_weighted, 
        'F1_Weighted': f1_weighted
    }


for archivo in os.listdir(ruta_carpeta_estudios):
    if archivo.endswith(".pkl"):  
        print(archivo)
        estudio = joblib.load(f"{ruta_carpeta_estudios}/{archivo}")

        mejores_hiperparametros = estudio.best_params
        print(mejores_hiperparametros)
        
        algorith = archivo.replace(".pkl", "")
        modelo = None
        scaler = None
        if algorith in scaler_config:
            scaler_name = scaler_config[algorith]
            scaler = eval(scaler_name)()
            X_train_scaled = scaler.fit_transform(X_train)
        else:
            scaler = StandardScaler()
            X_train_scaled = scaler.fit_transform(X_train)
        print(scaler)
        
        if algorith == "LinearRegression":
            modelo = LinearRegression(**mejores_hiperparametros)
        elif algorith == "Lasso":
            modelo = Lasso(**mejores_hiperparametros)
        elif algorith == "Ridge":
            modelo = Ridge(**mejores_hiperparametros)
        elif algorith == "ElasticNet":
            modelo = ElasticNet(**mejores_hiperparametros)
        elif algorith == "GradientBoostingRegressor":
            modelo = GradientBoostingRegressor(**mejores_hiperparametros)
        elif algorith == "HistGradientBoostingRegressor":
            modelo = HistGradientBoostingRegressor(**mejores_hiperparametros)
        elif algorith == "LGBMRegressor":
            modelo = LGBMRegressor(**mejores_hiperparametros)
        elif algorith == "XGBRegressor":
            modelo = XGBRegressor(**mejores_hiperparametros)
        elif algorith == "RandomForestRegressor":
            modelo = RandomForestRegressor(**mejores_hiperparametros)
        elif algorith == "KNeighborsRegressor":
            modelo = KNeighborsRegressor(**mejores_hiperparametros)
        elif algorith == "LinearSVR":
            modelo = LinearSVR(**mejores_hiperparametros)
        elif algorith == "XGBClassifier":
            modelo = XGBClassifier(**mejores_hiperparametros)
        elif algorith == "RandomForestClassifier":
            modelo = RandomForestClassifier(**mejores_hiperparametros)
        elif algorith == "HistGradientBoostingClassifier":
            modelo = HistGradientBoostingClassifier(**mejores_hiperparametros)
        elif algorith == "DecisionTreeClassifier":
            modelo = DecisionTreeClassifier(**mejores_hiperparametros)
        elif algorith == "LinearSVC":
            modelo = LinearSVC(**mejores_hiperparametros)
        
        if modelo is None:
            continue
        
        print('Entrenando modelo')
        modelo.fit(X_train_scaled, y_train)
        print('Modelo entrenado')


        if 'Classifier' in algorith or algorith == "LinearSVC":
            metricas = evaluar_modelo_clasificacion(algorith, modelo, X_test, y_test, scaler)
            metricas_clasificacion = pd.concat([metricas_clasificacion, pd.DataFrame([metricas])], ignore_index=True)
        else:
            metricas = evaluar_modelo_regresion(algorith, modelo, X_test, y_test, scaler)
            metricas_regresion = pd.concat([metricas_regresion, pd.DataFrame([metricas])], ignore_index=True)

                
        if not os.path.exists('train-test_models/5-bins-uniform'):
            os.makedirs('train-test_models/5-bins-uniform')
        ruta_guardado_modelo = f"train-test_models/5-bins-uniform/{archivo}"  
        joblib.dump(modelo, ruta_guardado_modelo)
        print(f"Modelo guardado en {ruta_guardado_modelo}")


metricas_regresion.to_csv('train-test_models/5-bins-uniform/resultados_metricas_regresion.csv', index=False)
metricas_clasificacion.to_csv('train-test_models/5-bins-uniform/resultados_metricas_clasificacion.csv', index=False)
print('Métricas de regresión guardadas en resultados_metricas_regresion.csv')
print('Métricas de clasificación guardadas en resultados_metricas_clasificacion.csv')

/home/jsolis/.local/lib/python3.11/site-packages/sklearn/preprocessing/_discretization.py:248: FutureWarning: In version 1.5 onwards, subsample=200_000 will be used by default. Set subsample explicitly to silence this warning in the mean time. Set subsample=None to disable subsampling explicitly.
  warnings.warn(


HistGradientBoostingClassifier.pkl
{'max_iter': 157, 'max_depth': 170, 'learning_rate': 0.17157978546124308, 'max_leaf_nodes': 5, 'class_weight': 'balanced'}
StandardScaler()
Entrenando modelo
Modelo entrenado


/tmp/ipykernel_8167/2443772160.py:217: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  metricas_clasificacion = pd.concat([metricas_clasificacion, pd.DataFrame([metricas])], ignore_index=True)


Modelo guardado en train-test_models/5-bins-uniform/HistGradientBoostingClassifier.pkl
LinearSVC.pkl
{'max_iter': 4134, 'C': 0.3409623247718362, 'class_weight': 'balanced', 'dual': 'auto'}
StandardScaler()
Entrenando modelo
Modelo entrenado
Modelo guardado en train-test_models/5-bins-uniform/LinearSVC.pkl
XGBClassifier.pkl
{'n_estimators': 127, 'max_depth': 60, 'learning_rate': 0.22003904520154657, 'max_leaves': 193, 'grow_policy': 'lossguide'}
StandardScaler()
Entrenando modelo
Modelo entrenado
Modelo guardado en train-test_models/5-bins-uniform/XGBClassifier.pkl
RandomForestClassifier.pkl
{'n_estimators': 62, 'max_depth': 6, 'max_features': 'log2', 'class_weight': 'balanced'}
StandardScaler()
Entrenando modelo
Modelo entrenado
Modelo guardado en train-test_models/5-bins-uniform/RandomForestClassifier.pkl
DecisionTreeClassifier.pkl
{'max_depth': 8, 'max_features': None, 'class_weight': 'balanced', 'splitter': 'best'}
StandardScaler()
Entrenando modelo
Modelo entrenado
Modelo guardado 

In [22]:
import pandas as pd
import numpy as np
import os
import joblib
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.ensemble import GradientBoostingRegressor, HistGradientBoostingRegressor, RandomForestRegressor, RandomForestClassifier
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor, XGBClassifier
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import LinearSVR, LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, explained_variance_score, mean_absolute_percentage_error
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, balanced_accuracy_score
from sklearn.preprocessing import StandardScaler, RobustScaler, PowerTransformer, Normalizer, MinMaxScaler, MaxAbsScaler, QuantileTransformer
from sklearn.metrics import classification_report, confusion_matrix


df = pd.read_csv('docs/600k_tracks_anonimous.csv')

ruta_carpeta_estudios = "jobs/10-bins-uniform"

X_train, X_test, y_train, y_test = discretize_and_split_data_train_test(df, n_bins=10, strategy='uniform')

def load_scaler_config(scaler_config_file):
    scaler_config = {}
    if os.path.exists(scaler_config_file):
        with open(scaler_config_file, 'r') as csvfile:
            reader = csv.reader(csvfile)
            for row in reader:
                if len(row) >= 2:
                    scaler_config[row[0]] = row[1]
                else:
                    print("Error: Hay lineas en blanco:", row)
    return scaler_config


scaler_config_file = 'logs/scaler_config.csv'
scaler_config = load_scaler_config(scaler_config_file)

metricas_regresion = pd.DataFrame(columns=['Modelo', 'MSE', 'RMSE', 'MAE', 'MAPE', 'R2', 'Explained_Variance'])
metricas_clasificacion = pd.DataFrame(columns=['Modelo', 'Balanced_Accuracy', 'Accuracy', 'Precision_Macro', 'Recall_Macro', 'F1_Macro', 'Precision_Weighted', 'Recall_Weighted', 'F1_Weighted'])

def evaluar_modelo_regresion(nombre_modelo, modelo, X_test, y_test, scaler):
    X_test_scaled = scaler.transform(X_test)
    y_pred = modelo.predict(X_test_scaled)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    explained_variance = explained_variance_score(y_test, y_pred)
    mape = mean_absolute_percentage_error(y_test, y_pred)
    
    if r2 < 0:
        r2 = 0
    if explained_variance < 0:
        explained_variance = 0
    
    return {'Modelo': nombre_modelo, 'MSE': mse, 'RMSE': rmse, 'MAE': mae, 'R2': r2, 'Explained_Variance': explained_variance, 'MAPE': mape}


def evaluar_modelo_clasificacion(nombre_modelo, modelo, X_test, y_test, scaler):
    X_test_scaled = scaler.transform(X_test)
    y_pred = modelo.predict(X_test_scaled)
    balanced_acc = balanced_accuracy_score(y_test, y_pred)
    accuracy = accuracy_score(y_test, y_pred)
    precision_macro = precision_score(y_test, y_pred, average='macro')
    recall_macro = recall_score(y_test, y_pred, average='macro')
    f1_macro = f1_score(y_test, y_pred, average='macro')
    precision_weighted = precision_score(y_test, y_pred, average='weighted')
    recall_weighted = recall_score(y_test, y_pred, average='weighted')
    f1_weighted = f1_score(y_test, y_pred, average='weighted')
    
    
    return {
        'Modelo': nombre_modelo, 
        'Balanced_Accuracy': balanced_acc, 
        'Accuracy': accuracy, 
        'Precision_Macro': precision_macro, 
        'Recall_Macro': recall_macro, 
        'F1_Macro': f1_macro, 
        'Precision_Weighted': precision_weighted, 
        'Recall_Weighted': recall_weighted, 
        'F1_Weighted': f1_weighted
    }


for archivo in os.listdir(ruta_carpeta_estudios):
    if archivo.endswith(".pkl"): 
        print(archivo)
        estudio = joblib.load(f"{ruta_carpeta_estudios}/{archivo}")

        mejores_hiperparametros = estudio.best_params
        print(mejores_hiperparametros)
        
        algorith = archivo.replace(".pkl", "")
        modelo = None
        scaler = None
        if algorith in scaler_config:
            scaler_name = scaler_config[algorith]
            scaler = eval(scaler_name)()
            X_train_scaled = scaler.fit_transform(X_train)
        else:
            scaler = StandardScaler()
            X_train_scaled = scaler.fit_transform(X_train)
        print(scaler)
        
        if algorith == "LinearRegression":
            modelo = LinearRegression(**mejores_hiperparametros)
        elif algorith == "Lasso":
            modelo = Lasso(**mejores_hiperparametros)
        elif algorith == "Ridge":
            modelo = Ridge(**mejores_hiperparametros)
        elif algorith == "ElasticNet":
            modelo = ElasticNet(**mejores_hiperparametros)
        elif algorith == "GradientBoostingRegressor":
            modelo = GradientBoostingRegressor(**mejores_hiperparametros)
        elif algorith == "HistGradientBoostingRegressor":
            modelo = HistGradientBoostingRegressor(**mejores_hiperparametros)
        elif algorith == "LGBMRegressor":
            modelo = LGBMRegressor(n_jobs = 20, **mejores_hiperparametros)
        elif algorith == "XGBRegressor":
            modelo = XGBRegressor(n_jobs = 20, **mejores_hiperparametros)
        elif algorith == "RandomForestRegressor":
            modelo = RandomForestRegressor(n_jobs = 20, **mejores_hiperparametros)
        elif algorith == "KNeighborsRegressor":
            modelo = KNeighborsRegressor(n_jobs = 20, **mejores_hiperparametros)
        elif algorith == "LinearSVR":
            modelo = LinearSVR(**mejores_hiperparametros)
        elif algorith == "XGBClassifier":
            modelo = XGBClassifier(n_jobs = 20, **mejores_hiperparametros)
        elif algorith == "RandomForestClassifier":
            modelo = RandomForestClassifier(n_jobs = 20, **mejores_hiperparametros)
        elif algorith == "HistGradientBoostingClassifier":
            modelo = HistGradientBoostingClassifier(**mejores_hiperparametros)
        elif algorith == "DecisionTreeClassifier":
            modelo = DecisionTreeClassifier(**mejores_hiperparametros)
        elif algorith == "LinearSVC":
            modelo = LinearSVC(**mejores_hiperparametros)
        if modelo is None:
            continue
        
        print('Entrenando modelo')
        modelo.fit(X_train_scaled, y_train)
        print('Modelo entrenado')

        if 'Classifier' in algorith or algorith == "LinearSVC":
            metricas = evaluar_modelo_clasificacion(algorith, modelo, X_test, y_test, scaler)
            metricas_clasificacion = pd.concat([metricas_clasificacion, pd.DataFrame([metricas])], ignore_index=True)
        else:
            metricas = evaluar_modelo_regresion(algorith, modelo, X_test, y_test, scaler)
            metricas_regresion = pd.concat([metricas_regresion, pd.DataFrame([metricas])], ignore_index=True)

        y_pred = modelo.predict(X_test)
        conf_matrix = confusion_matrix(y_test, y_pred)
        print("Matriz de confusión:")
        print(conf_matrix)
        
        class_report = classification_report(y_test, y_pred)
        print("Reporte de clasificación:")
        print(class_report)
                
        if not os.path.exists('train-test_models/10-bins-uniform'):
            os.makedirs('train-test_models/10-bins-uniform')
        ruta_guardado_modelo = f"train-test_models/10-bins-uniform/{archivo}" 
        joblib.dump(modelo, ruta_guardado_modelo)
        print(f"Modelo guardado en {ruta_guardado_modelo}")

metricas_regresion.to_csv('train-test_models/10-bins-uniform/resultados_metricas_regresion.csv', index=False)
metricas_clasificacion.to_csv('train-test_models/10-bins-uniform/resultados_metricas_clasificacion.csv', index=False)
print('Métricas de regresión guardadas en resultados_metricas_regresion.csv')
print('Métricas de clasificación guardadas en resultados_metricas_clasificacion.csv')

/home/jsolis/.local/lib/python3.11/site-packages/sklearn/preprocessing/_discretization.py:248: FutureWarning: In version 1.5 onwards, subsample=200_000 will be used by default. Set subsample explicitly to silence this warning in the mean time. Set subsample=None to disable subsampling explicitly.
  warnings.warn(


HistGradientBoostingClassifier.pkl
{'max_iter': 307, 'max_depth': 4, 'learning_rate': 0.029218637681534418, 'max_leaf_nodes': 12, 'class_weight': 'balanced'}
StandardScaler()
Entrenando modelo
Modelo entrenado


/tmp/ipykernel_8167/1598872176.py:157: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  metricas_clasificacion = pd.concat([metricas_clasificacion, pd.DataFrame([metricas])], ignore_index=True)


Matriz de confusión:
[[36705  3054   190   177     5     0     0     0     0     0]
 [27706  1372    32    19     4     0     0     0     0     0]
 [34532  1746    66    20     6     0     0     0     0     0]
 [34025  1798    93    32     2     0     0     0     0     0]
 [25437  1424    94    34     1     0     0     0     0     0]
 [15046   725    49    18     2     0     0     0     0     0]
 [ 6476   268    18     7     3     0     0     0     0     0]
 [ 2046    52     3     1     0     0     0     0     0     0]
 [  294     4     0     0     0     0     0     0     0     0]
 [   16     0     0     0     0     0     0     0     0     0]]


/home/jsolis/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/jsolis/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/jsolis/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


...

Reporte de clasificación:
              precision    recall  f1-score   support

         0.0       0.79      0.01      0.02     40131
         1.0       0.14      0.48      0.22     29133
         2.0       0.18      0.46      0.26     36370
         3.0       0.15      0.00      0.01     35950
         4.0       0.00      0.00      0.00     26990
         5.0       0.00      0.00      0.00     15840
         6.0       0.00      0.00      0.00      6772
         7.0       0.00      0.00      0.00      2102
         8.0       0.00      0.00      0.00       298
         9.0       0.00      0.00      0.00        16

    accuracy                           0.16    193602
   macro avg       0.13      0.10      0.05    193602
weighted avg       0.25      0.16      0.09    193602

Modelo guardado en train-test_models/10-bins-uniform/XGBClassifier.pkl
RandomForestClassifier.pkl
{'n_estimators': 60, 'max_depth': 4, 'max_features': 'log2', 'class_weight': 'balanced'}
StandardScaler()
Entrenando 

/home/jsolis/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/jsolis/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/jsolis/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Reporte de clasificación:
              precision    recall  f1-score   support

         0.0       0.44      0.64      0.52     40131
         1.0       0.18      0.34      0.24     29133
         2.0       0.19      0.42      0.26     36370
         3.0       0.00      0.00      0.00     35950
         4.0       0.09      0.00      0.00     26990
         5.0       0.00      0.00      0.00     15840
         6.0       0.00      0.00      0.00      6772
         7.0       0.00      0.00      0.00      2102
         8.0       0.00      0.00      0.00       298
         9.0       0.00      0.00      0.00        16

    accuracy                           0.26    193602
   macro avg       0.09      0.14      0.10    193602
weighted avg       0.17      0.26      0.19    193602

Modelo guardado en train-test_models/10-bins-uniform/RandomForestClassifier.pkl
DecisionTreeClassifier.pkl
{'max_depth': 6, 'max_features': 'log2', 'class_weight': 'balanced', 'splitter': 'best'}
StandardScaler()
En

/home/jsolis/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/jsolis/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/jsolis/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Reporte de clasificación:
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00     40131
         1.0       0.00      0.00      0.00     29133
         2.0       0.00      0.00      0.00     36370
         3.0       0.00      0.00      0.00     35950
         4.0       0.14      0.99      0.24     26990
         5.0       0.17      0.02      0.03     15840
         6.0       0.00      0.00      0.00      6772
         7.0       0.00      0.00      0.00      2102
         8.0       0.00      0.00      0.00       298
         9.0       0.00      0.00      0.00        16

    accuracy                           0.14    193602
   macro avg       0.03      0.10      0.03    193602
weighted avg       0.03      0.14      0.04    193602

Modelo guardado en train-test_models/10-bins-uniform/DecisionTreeClassifier.pkl
Métricas de regresión guardadas en resultados_metricas_regresion.csv
Métricas de clasificación guardadas en resultados_metricas_clasifi

Validación Cruzada

In [17]:
from sklearn.model_selection import KFold, StratifiedKFold


df = pd.read_csv('docs/600k_tracks_anonimous.csv')

features = df.drop('popularity', axis=1)
target = df['popularity']

imputer = SimpleImputer(strategy='mean')
features = imputer.fit_transform(features)

ruta_carpeta_estudios = "jobs"

def split_data_cv(features, target, problem_type, n_splits=5, random_state=42):
    if problem_type == 'regression':
        cv = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    elif problem_type == 'classification':
        cv = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    
    for train_index, test_index in cv.split(features, target):
        X_train, X_test = features[train_index], features[test_index]
        y_train, y_test = target[train_index], target[test_index]
        yield X_train, X_test, y_train, y_test

for archivo in os.listdir(ruta_carpeta_estudios):
    if archivo.endswith(".pkl"):  
        print(archivo)
        estudio = joblib.load(f"{ruta_carpeta_estudios}/{archivo}")

        mejores_hiperparametros = estudio.best_params
        print(mejores_hiperparametros)
        
        algorith = archivo.replace(".pkl", "")
        modelo = None
        scaler = None
        if algorith in scaler_config:
            scaler_name = scaler_config[algorith]
            scaler = eval(scaler_name)()
        else:
            scaler = StandardScaler()
        print(scaler)
        
        if algorith == "LinearRegression":
            modelo = LinearRegression(**mejores_hiperparametros)
            problem_type = 'regression'
        elif algorith == "Lasso":
            modelo = Lasso(**mejores_hiperparametros)
            problem_type = 'regression'
        elif algorith == "Ridge":
            modelo = Ridge(**mejores_hiperparametros)
            problem_type = 'regression'
        elif algorith == "ElasticNet":
            modelo = ElasticNet(**mejores_hiperparametros)
            problem_type = 'regression'
        elif algorith == "GradientBoostingRegressor":
            modelo = GradientBoostingRegressor(**mejores_hiperparametros)
            problem_type = 'regression'
        elif algorith == "HistGradientBoostingRegressor":
            modelo = HistGradientBoostingRegressor(**mejores_hiperparametros)
            problem_type = 'regression'
        elif algorith == "LGBMRegressor":
            modelo = LGBMRegressor(n_jobs = 20, **mejores_hiperparametros)
            problem_type = 'regression'
        elif algorith == "XGBRegressor":
            modelo = XGBRegressor(n_jobs = 20, **mejores_hiperparametros)
            problem_type = 'regression'
        elif algorith == "RandomForestRegressor":
            modelo = RandomForestRegressor(n_jobs = 20, **mejores_hiperparametros)
            problem_type = 'regression'
        elif algorith == "KNeighborsRegressor":
            modelo = KNeighborsRegressor(n_jobs = 20, **mejores_hiperparametros)
            problem_type = 'regression'
        elif algorith == "LinearSVR":
            modelo = LinearSVR(**mejores_hiperparametros)
            problem_type = 'regression'
        elif algorith == "XGBClassifier":
            modelo = XGBClassifier(n_jobs = 20, **mejores_hiperparametros)
            problem_type = 'classification'
        elif algorith == "RandomForestClassifier":
            modelo = RandomForestClassifier(n_jobs = 20, **mejores_hiperparametros)
            problem_type = 'classification'
        elif algorith == "HistGradientBoostingClassifier":
            modelo = HistGradientBoostingClassifier(**mejores_hiperparametros)
            problem_type = 'classification'
        elif algorith == "DecisionTreeClassifier":
            modelo = DecisionTreeClassifier(**mejores_hiperparametros)
            problem_type = 'classification'
        elif algorith == "LinearSVC":
            modelo = LinearSVC(**mejores_hiperparametros)
            problem_type = 'classification'
        if modelo is None:
            continue
        
        print('Entrenando modelo')
        
        for X_train, X_test, y_train, y_test in split_data_cv(features, target, problem_type):
            X_train_scaled = scaler.fit_transform(X_train)
            X_test_scaled = scaler.transform(X_test)
            
            modelo.fit(X_train_scaled, y_train)
            
            if problem_type == 'regression':
                metricas = evaluar_modelo_regresion(algorith, modelo, X_test, y_test, scaler)
                metricas_regresion = pd.concat([metricas_regresion, pd.DataFrame([metricas])], ignore_index=True)
            elif problem_type == 'classification':
                metricas = evaluar_modelo_clasificacion(algorith, modelo, X_test, y_test, scaler)
                metricas_clasificacion = pd.concat([metricas_clasificacion, pd.DataFrame([metricas])], ignore_index=True)
        
        print('Modelo entrenado')

        if not os.path.exists('cross_val_models'):
            os.makedirs('cross_val_models')
        ruta_guardado_modelo = f"cross_val_models/{archivo}" 
        joblib.dump(modelo, ruta_guardado_modelo)
        print(f"Modelo guardado en {ruta_guardado_modelo}")

metricas_regresion.to_csv('cross_val_models/resultados_metricas_regresion.csv', index=False)
metricas_clasificacion.to_csv('cross_val_models/resultados_metricas_clasificacion.csv', index=False)
print('Métricas de regresión guardadas en resultados_metricas_regresion.csv')
print('Métricas de clasificación guardadas en resultados_metricas_clasificacion.csv')


LinearSVR.pkl
{'dual': 'auto', 'C': 0.14290067275948565, 'max_iter': 95591}
StandardScaler()
Entrenando modelo
Modelo entrenado
Modelo guardado en cross_val_models/LinearSVR.pkl
KNeighborsRegressor.pkl
{'n_neighbors': 69, 'weights': 'distance', 'algorithm': 'brute', 'p': 1}
StandardScaler()
Entrenando modelo
Modelo entrenado
Modelo guardado en cross_val_models/KNeighborsRegressor.pkl
Ridge.pkl
{'alpha': 0.02613865143852783, 'solver': 'cholesky', 'max_iter': 14801}
StandardScaler()
Entrenando modelo
Modelo entrenado
Modelo guardado en cross_val_models/Ridge.pkl
LGBMRegressor.pkl
{'n_estimators': 118, 'max_depth': 10, 'learning_rate': 0.15859769003466326, 'num_leaves': 57}
StandardScaler()
Entrenando modelo
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.008796 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2572
[LightGBM] [Info] Number 

In [18]:
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import classification_report, confusion_matrix


df = pd.read_csv('docs/600k_tracks_anonimous.csv')


def split_data_cv(features, target, problem_type, n_splits=5, random_state=42):
    if problem_type == 'regression':
        cv = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    elif problem_type == 'classification':
        cv = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    
    for train_index, test_index in cv.split(features, target):
        X_train, X_test = features[train_index], features[test_index]
        y_train, y_test = target[train_index], target[test_index]
        yield X_train, X_test, y_train, y_test


ruta_carpeta_estudios = "jobs/5-bins-uniform"

X_train, y_train = discretize_and_split_data(df, n_bins=5, strategy='uniform')

features = X_train
target = y_train

for archivo in os.listdir(ruta_carpeta_estudios):
    if archivo.endswith(".pkl"):  
        print(archivo)
        estudio = joblib.load(f"{ruta_carpeta_estudios}/{archivo}")

        mejores_hiperparametros = estudio.best_params
        print(mejores_hiperparametros)
        
        algorith = archivo.replace(".pkl", "")
        modelo = None
        scaler = None
        if algorith in scaler_config:
            scaler_name = scaler_config[algorith]
            scaler = eval(scaler_name)()
        else:
            scaler = StandardScaler()
        
        if algorith == "LinearRegression":
            modelo = LinearRegression(**mejores_hiperparametros)
            problem_type = 'regression'
        elif algorith == "Lasso":
            modelo = Lasso(**mejores_hiperparametros)
            problem_type = 'regression'
        elif algorith == "Ridge":
            modelo = Ridge(**mejores_hiperparametros)
            problem_type = 'regression'
        elif algorith == "ElasticNet":
            modelo = ElasticNet(**mejores_hiperparametros)
            problem_type = 'regression'
        elif algorith == "GradientBoostingRegressor":
            modelo = GradientBoostingRegressor(**mejores_hiperparametros)
            problem_type = 'regression'
        elif algorith == "HistGradientBoostingRegressor":
            modelo = HistGradientBoostingRegressor(**mejores_hiperparametros)
            problem_type = 'regression'
        elif algorith == "LGBMRegressor":
            modelo = LGBMRegressor(n_jobs = 20, **mejores_hiperparametros)
            problem_type = 'regression'
        elif algorith == "XGBRegressor":
            modelo = XGBRegressor(n_jobs = 20, **mejores_hiperparametros)
            problem_type = 'regression'
        elif algorith == "RandomForestRegressor":
            modelo = RandomForestRegressor(n_jobs = 20, **mejores_hiperparametros)
            problem_type = 'regression'
        elif algorith == "KNeighborsRegressor":
            modelo = KNeighborsRegressor(n_jobs = 20, **mejores_hiperparametros)
            problem_type = 'regression'
        elif algorith == "LinearSVR":
            modelo = LinearSVR(**mejores_hiperparametros)
            problem_type = 'regression'
        elif algorith == "XGBClassifier":
            modelo = XGBClassifier(n_jobs = 20, **mejores_hiperparametros)
            problem_type = 'classification'
        elif algorith == "RandomForestClassifier":
            modelo = RandomForestClassifier(n_jobs = 20, **mejores_hiperparametros)
            problem_type = 'classification'
        elif algorith == "HistGradientBoostingClassifier":
            modelo = HistGradientBoostingClassifier(**mejores_hiperparametros)
            problem_type = 'classification'
        elif algorith == "DecisionTreeClassifier":
            modelo = DecisionTreeClassifier(**mejores_hiperparametros)
            problem_type = 'classification'
        elif algorith == "LinearSVC":
            modelo = LinearSVC(**mejores_hiperparametros)
            problem_type = 'classification'
        if modelo is None:
            continue
        
        print('Entrenando modelo')
        
        problem_type = 'classification' if 'Classifier' in algorith or algorith == "LinearSVC" else 'regression'
        for X_train, X_test, y_train, y_test in split_data_cv(features, target, problem_type):
            X_train_scaled = scaler.fit_transform(X_train)
            X_test_scaled = scaler.transform(X_test)
            
            modelo.fit(X_train_scaled, y_train)
            
            if problem_type == 'regression':
                metricas = evaluar_modelo_regresion(algorith, modelo, X_test, y_test, scaler)
                metricas_regresion = pd.concat([metricas_regresion, pd.DataFrame([metricas])], ignore_index=True)
            elif problem_type == 'classification':
                metricas = evaluar_modelo_clasificacion(algorith, modelo, X_test, y_test, scaler)
                metricas_clasificacion = pd.concat([metricas_clasificacion, pd.DataFrame([metricas])], ignore_index=True)

        y_pred = modelo.predict(X_test)
        
        conf_matrix = confusion_matrix(y_test, y_pred)
        print("Matriz de confusión:")
        print(conf_matrix)
        
        class_report = classification_report(y_test, y_pred)
        print("Reporte de clasificación:")
        print(class_report)
                
        if not os.path.exists('train-test_models/5-bins-uniform'):
            os.makedirs('train-test_models/5-bins-uniform')
        ruta_guardado_modelo = f"cross_val_models/5-bins-uniform/{archivo}"  
        joblib.dump(modelo, ruta_guardado_modelo)
        print(f"Modelo guardado en {ruta_guardado_modelo}")

metricas_regresion.to_csv('cross_val_models/5-bins-uniform/resultados_metricas_regresion.csv', index=False)
metricas_clasificacion.to_csv('cross_val_models/5-bins-uniform/resultados_metricas_clasificacion.csv', index=False)
print('Métricas de regresión guardadas en resultados_metricas_regresion.csv')
print('Métricas de clasificación guardadas en resultados_metricas_clasificacion.csv')


/home/jsolis/.local/lib/python3.11/site-packages/sklearn/preprocessing/_discretization.py:248: FutureWarning: In version 1.5 onwards, subsample=200_000 will be used by default. Set subsample explicitly to silence this warning in the mean time. Set subsample=None to disable subsampling explicitly.
  warnings.warn(


HistGradientBoostingClassifier.pkl
{'max_iter': 157, 'max_depth': 170, 'learning_rate': 0.17157978546124308, 'max_leaf_nodes': 5, 'class_weight': 'balanced'}
Entrenando modelo
Matriz de confusión:
[[36061  5916     1     0     0]
 [32260 11569     1     0     0]
 [17142  8813     2     0     0]
 [ 3402  1974     2     0     0]
 [  121    70     0     0     0]]
Reporte de clasificación:
              precision    recall  f1-score   support

         0.0       0.41      0.86      0.55     41978
         1.0       0.41      0.26      0.32     43830
         2.0       0.33      0.00      0.00     25957
         3.0       0.00      0.00      0.00      5378
         4.0       0.00      0.00      0.00       191

    accuracy                           0.41    117334
   macro avg       0.23      0.22      0.17    117334
weighted avg       0.37      0.41      0.32    117334



/home/jsolis/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/jsolis/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/jsolis/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Modelo guardado en cross_val_models/5-bins-uniform/HistGradientBoostingClassifier.pkl
LinearSVC.pkl
{'max_iter': 4134, 'C': 0.3409623247718362, 'class_weight': 'balanced', 'dual': 'auto'}
Entrenando modelo
Matriz de confusión:
[[    0     0 41978     0     0]
 [    0     0 43830     0     0]
 [    0     0 25957     0     0]
 [    0     0  5378     0     0]
 [    0     0   191     0     0]]
Reporte de clasificación:
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00     41978
         1.0       0.00      0.00      0.00     43830
         2.0       0.22      1.00      0.36     25957
         3.0       0.00      0.00      0.00      5378
         4.0       0.00      0.00      0.00       191

    accuracy                           0.22    117334
   macro avg       0.04      0.20      0.07    117334
weighted avg       0.05      0.22      0.08    117334

Modelo guardado en cross_val_models/5-bins-uniform/LinearSVC.pkl
XGBClassifier.pkl


/home/jsolis/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/jsolis/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/jsolis/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


...

/home/jsolis/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/jsolis/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/jsolis/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


{'n_estimators': 62, 'max_depth': 6, 'max_features': 'log2', 'class_weight': 'balanced'}
Entrenando modelo
Matriz de confusión:
[[21481 20487    10     0     0]
 [ 9918 33905     7     0     0]
 [ 3631 22322     4     0     0]
 [  527  4847     4     0     0]
 [    9   182     0     0     0]]
Reporte de clasificación:
              precision    recall  f1-score   support

         0.0       0.60      0.51      0.55     41978
         1.0       0.41      0.77      0.54     43830
         2.0       0.16      0.00      0.00     25957
         3.0       0.00      0.00      0.00      5378
         4.0       0.00      0.00      0.00       191

    accuracy                           0.47    117334
   macro avg       0.24      0.26      0.22    117334
weighted avg       0.41      0.47      0.40    117334



/home/jsolis/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/jsolis/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/jsolis/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Modelo guardado en cross_val_models/5-bins-uniform/RandomForestClassifier.pkl
DecisionTreeClassifier.pkl
{'max_depth': 8, 'max_features': None, 'class_weight': 'balanced', 'splitter': 'best'}
Entrenando modelo
Matriz de confusión:
[[23182 18780    16     0     0]
 [11860 31961     9     0     0]
 [ 4567 21384     6     0     0]
 [  672  4702     4     0     0]
 [   14   177     0     0     0]]
Reporte de clasificación:
              precision    recall  f1-score   support

         0.0       0.58      0.55      0.56     41978
         1.0       0.42      0.73      0.53     43830
         2.0       0.17      0.00      0.00     25957
         3.0       0.00      0.00      0.00      5378
         4.0       0.00      0.00      0.00       191

    accuracy                           0.47    117334
   macro avg       0.23      0.26      0.22    117334
weighted avg       0.40      0.47      0.40    117334

Modelo guardado en cross_val_models/5-bins-uniform/DecisionTreeClassifier.pkl
Métricas d

/home/jsolis/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/jsolis/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/jsolis/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [19]:
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import classification_report, confusion_matrix


df = pd.read_csv('docs/600k_tracks_anonimous.csv')

def split_data_cv(features, target, problem_type, n_splits=5, random_state=42):
    if problem_type == 'regression':
        cv = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    elif problem_type == 'classification':
        cv = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    
    for train_index, test_index in cv.split(features, target):
        X_train, X_test = features[train_index], features[test_index]
        y_train, y_test = target[train_index], target[test_index]
        yield X_train, X_test, y_train, y_test


ruta_carpeta_estudios = "jobs/10-bins-uniform"

X_train, y_train = discretize_and_split_data(df, n_bins=10, strategy='uniform')

features = X_train
target = y_train

for archivo in os.listdir(ruta_carpeta_estudios):
    if archivo.endswith(".pkl"): 
        print(archivo)
        estudio = joblib.load(f"{ruta_carpeta_estudios}/{archivo}")

        mejores_hiperparametros = estudio.best_params
        print(mejores_hiperparametros)

        algorith = archivo.replace(".pkl", "")
        modelo = None
        scaler = None
        if algorith in scaler_config:
            scaler_name = scaler_config[algorith]
            scaler = eval(scaler_name)()
        else:
            scaler = StandardScaler()
        
        if algorith == "LinearRegression":
            modelo = LinearRegression(**mejores_hiperparametros)
            problem_type = 'regression'
        elif algorith == "Lasso":
            modelo = Lasso(**mejores_hiperparametros)
            problem_type = 'regression'
        elif algorith == "Ridge":
            modelo = Ridge(**mejores_hiperparametros)
            problem_type = 'regression'
        elif algorith == "ElasticNet":
            modelo = ElasticNet(**mejores_hiperparametros)
            problem_type = 'regression'
        elif algorith == "GradientBoostingRegressor":
            modelo = GradientBoostingRegressor(**mejores_hiperparametros)
            problem_type = 'regression'
        elif algorith == "HistGradientBoostingRegressor":
            modelo = HistGradientBoostingRegressor(**mejores_hiperparametros)
            problem_type = 'regression'
        elif algorith == "LGBMRegressor":
            modelo = LGBMRegressor(n_jobs = 20, **mejores_hiperparametros)
            problem_type = 'regression'
        elif algorith == "XGBRegressor":
            modelo = XGBRegressor(n_jobs = 20, **mejores_hiperparametros)
            problem_type = 'regression'
        elif algorith == "RandomForestRegressor":
            modelo = RandomForestRegressor(n_jobs = 20, **mejores_hiperparametros)
            problem_type = 'regression'
        elif algorith == "KNeighborsRegressor":
            modelo = KNeighborsRegressor(n_jobs = 20, **mejores_hiperparametros)
            problem_type = 'regression'
        elif algorith == "LinearSVR":
            modelo = LinearSVR(**mejores_hiperparametros)
            problem_type = 'regression'
        elif algorith == "XGBClassifier":
            modelo = XGBClassifier(n_jobs = 20, **mejores_hiperparametros)
            problem_type = 'classification'
        elif algorith == "RandomForestClassifier":
            modelo = RandomForestClassifier(n_jobs = 20, **mejores_hiperparametros)
            problem_type = 'classification'
        elif algorith == "HistGradientBoostingClassifier":
            modelo = HistGradientBoostingClassifier(**mejores_hiperparametros)
            problem_type = 'classification'
        elif algorith == "DecisionTreeClassifier":
            modelo = DecisionTreeClassifier(**mejores_hiperparametros)
            problem_type = 'classification'
        elif algorith == "LinearSVC":
            modelo = LinearSVC(**mejores_hiperparametros)
            problem_type = 'classification'
        if modelo is None:
            continue
        
        print('Entrenando modelo')
        
        problem_type = 'classification' if 'Classifier' in algorith or algorith == "LinearSVC" else 'regression'
        for X_train, X_test, y_train, y_test in split_data_cv(features, target, problem_type):
            X_train_scaled = scaler.fit_transform(X_train)
            X_test_scaled = scaler.transform(X_test)
            
            modelo.fit(X_train_scaled, y_train)
            
            if problem_type == 'regression':
                metricas = evaluar_modelo_regresion(algorith, modelo, X_test, y_test, scaler)
                metricas_regresion = pd.concat([metricas_regresion, pd.DataFrame([metricas])], ignore_index=True)
            elif problem_type == 'classification':
                metricas = evaluar_modelo_clasificacion(algorith, modelo, X_test, y_test, scaler)
                metricas_clasificacion = pd.concat([metricas_clasificacion, pd.DataFrame([metricas])], ignore_index=True)

        y_pred = modelo.predict(X_test)
        
        conf_matrix = confusion_matrix(y_test, y_pred)
        print("Matriz de confusión:")
        print(conf_matrix)
        
        class_report = classification_report(y_test, y_pred)
        print("Reporte de clasificación:")
        print(class_report)
                
        if not os.path.exists('train-test_models/10-bins-uniform'):
            os.makedirs('train-test_models/10-bins-uniform')
        ruta_guardado_modelo = f"cross_val_models/10-bins-uniform/{archivo}"  
        joblib.dump(modelo, ruta_guardado_modelo)
        print(f"Modelo guardado en {ruta_guardado_modelo}")

metricas_regresion.to_csv('cross_val_models/10-bins-uniform/resultados_metricas_regresion.csv', index=False)
metricas_clasificacion.to_csv('cross_val_models/10-bins-uniform/resultados_metricas_clasificacion.csv', index=False)
print('Métricas de regresión guardadas en resultados_metricas_regresion.csv')
print('Métricas de clasificación guardadas en resultados_metricas_clasificacion.csv')


/home/jsolis/.local/lib/python3.11/site-packages/sklearn/preprocessing/_discretization.py:248: FutureWarning: In version 1.5 onwards, subsample=200_000 will be used by default. Set subsample explicitly to silence this warning in the mean time. Set subsample=None to disable subsampling explicitly.
  warnings.warn(


HistGradientBoostingClassifier.pkl
{'max_iter': 307, 'max_depth': 4, 'learning_rate': 0.029218637681534418, 'max_leaf_nodes': 12, 'class_weight': 'balanced'}
Entrenando modelo
Matriz de confusión:
[[23308   944    59     5     1     4     0     0     0     0]
 [17327   308    17     3     1     0     0     0     0     0]
 [21716   282    38     4     0     2     0     0     0     0]
 [21499   221    62     6     0     0     0     0     0     0]
 [16158   145    44    11     0     0     0     0     0     0]
 [ 9463    79    46    11     0     1     0     0     0     0]
 [ 4054    30    17     3     0     0     0     0     0     0]
 [ 1255     9     9     0     0     1     0     0     0     0]
 [  179     1     1     0     0     0     0     0     0     0]
 [   10     0     0     0     0     0     0     0     0     0]]
Reporte de clasificación:
              precision    recall  f1-score   support

         0.0       0.20      0.96      0.33     24321
         1.0       0.15      0.02    

/home/jsolis/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/jsolis/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/jsolis/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Modelo guardado en cross_val_models/10-bins-uniform/HistGradientBoostingClassifier.pkl
LinearSVC.pkl
{'max_iter': 30143, 'C': 0.4435817075441573, 'class_weight': 'balanced', 'dual': 'auto'}
Entrenando modelo
Matriz de confusión:
[[    0     0     0     0 24321     0     0     0     0     0]
 [    0     0     0     0 17656     0     0     0     0     0]
 [    0     0     0     0 22042     0     0     0     0     0]
 [    0     0     0     0 21788     0     0     0     0     0]
 [    0     0     0     0 16358     0     0     0     0     0]
 [    0     0     0     0  9600     0     0     0     0     0]
 [    0     0     0     0  4104     0     0     0     0     0]
 [    0     0     0     0  1274     0     0     0     0     0]
 [    0     0     0     0   181     0     0     0     0     0]
 [    0     0     0     0    10     0     0     0     0     0]]
Reporte de clasificación:
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00     24321
     

/home/jsolis/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/jsolis/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/jsolis/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


...

/home/jsolis/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/jsolis/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/jsolis/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


{'max_depth': 6, 'max_features': 'log2', 'class_weight': 'balanced', 'splitter': 'best'}
Entrenando modelo


/home/jsolis/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/jsolis/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/jsolis/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/jsolis/.local/li

Matriz de confusión:
[[24309     0     0     0     0     0    11     0     0     1]
 [17651     0     0     0     0     0     5     0     0     0]
 [22032     0     0     0     0     0     5     0     0     5]
 [21783     0     0     0     0     0     1     0     0     4]
 [16355     0     0     0     0     0     2     0     0     1]
 [ 9595     0     0     0     0     0     1     0     0     4]
 [ 4102     0     0     0     0     0     1     0     0     1]
 [ 1273     0     0     0     0     0     1     0     0     0]
 [  181     0     0     0     0     0     0     0     0     0]
 [   10     0     0     0     0     0     0     0     0     0]]
Reporte de clasificación:
              precision    recall  f1-score   support

         0.0       0.21      1.00      0.34     24321
         1.0       0.00      0.00      0.00     17656
         2.0       0.00      0.00      0.00     22042
         3.0       0.00      0.00      0.00     21788
         4.0       0.00      0.00      0.00     163

/home/jsolis/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/jsolis/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/jsolis/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [28]:
import pandas as pd
df = pd.read_csv('cross_val_models/resultados_metricas_regresion.csv')
excel_file = f'cross_val_models/resultados_metricas_regresion.xlsx'
df1 = df.groupby('Modelo', as_index=False).mean()
df1.to_excel(excel_file, index=False)
df = pd.read_csv('cross_val_models/5-bins-uniform/resultados_metricas_clasificacion.csv')
excel_file = f'cross_val_models/5-bins-uniform/resultados_metricas_clasificacion.xlsx'
df1 = df.groupby('Modelo', as_index=False).mean()
df1.to_excel(excel_file, index=False)
df = pd.read_csv('cross_val_models/10-bins-uniform/resultados_metricas_clasificacion.csv')
excel_file = f'cross_val_models/10-bins-uniform/resultados_metricas_clasificacion.xlsx'
df1 = df.groupby('Modelo', as_index=False).mean()
df1.to_excel(excel_file, index=False)